# 8. Resolve data issues

The analysis in section 8.2 is based on data retrieved from LexisNexis (LexisNexis Dossier).

In [1]:
sample_file = '../preprocessed_data/sample_2019-09-10.feather'
issues_to_address = '../preprocessed_data/issues_to_address_2019-09-10.pickle'
names_table_file = '../preprocessed_data/names_table_2019-09-09.feather'

pipelines_2010_raw = '../data/pipelines_2010_2019-08-11.feather'
pipelines_2004_raw = '../data/pipelines_2004_2019-08-11.feather'

## Setup

In [2]:
import pandas as pd
import numpy as np
from datetime import date
from functools import partial

today = date.today().isoformat()

## Load data

In [3]:
import pickle

with open(issues_to_address, 'rb') as file:
    issues = pickle.load(file)
    
issues = (issue for issue in issues)

In [4]:
sample = pd.read_feather(sample_file)
sample.sample(5)
assert sample.duplicated(subset=['OPERATOR_ID', 'YEAR', 'COMMODITY']).sum() == 0

In [5]:
names_table = pd.read_feather(names_table_file)
names_table['NAME'] = names_table['NAME'].str.title()
names_table['NAME'] = names_table['NAME'].str.replace(r'[^A-Za-z0-9\s]+', '')
names_table.sample(3)

,OPERATOR_ID,YEAR,NAME,STREET
6307,30755,2009,Citgo Products Pipeline Co,"6100 S. YALE, PO 3758"
4018,99031,2018,Citgo Petroleum Corporation Terminals,1293 ELDRIDGE PARKWAY
2750,32493,2013,Unev Pipeline Llc,"2828 N. HARWOOD ST., SUITE 1300"


In [6]:
pipelines_2010_raw = pd.read_feather(pipelines_2010_raw)
pipelines_2004_raw = pd.read_feather(pipelines_2004_raw)

## 8.1 Functions for analysis

In [7]:
from functools import partial

def find_info(OPERATOR_ID, info_col: str, title: str, df = names_table, id_col = 'OPERATOR_ID', 
              year_col = 'YEAR', fuzzy=False):
    from fuzzywuzzy import fuzz
    
    values = np.unique(df[df[id_col] == int(OPERATOR_ID)][info_col]).tolist()
    result = []
    for value in values:
        start_year = df[df[info_col] == value][year_col].min()
        end_year = df[df[info_col] == value][year_col].max()
        id_ = OPERATOR_ID
        result = result + [{title: value, 'start_year': start_year, 'end_year': end_year, 'id_': id_}]
        
    if fuzzy and len(result) == 2 and fuzz.ratio(result[0][title].lower(), result[1][title].lower()) >= 95:
            result = [result[0]]
            
    return(result)

find_address = partial(find_info, info_col='STREET', title='street')
find_name = partial(find_info, info_col='NAME', title='name', fuzzy=True)
find_name('3445')

[{'name': 'Dixie Pipeline',
  'start_year': 2007,
  'end_year': 2010,
  'id_': '3445'},
 {'name': 'Dixie Pipeline Company',
  'start_year': 2004,
  'end_year': 2006,
  'id_': '3445'},
 {'name': 'Dixie Pipeline Company Llc',
  'start_year': 2011,
  'end_year': 2018,
  'id_': '3445'}]

In [8]:
def report_names(names: list):
    for name in names: 
        hist = find_name(name)
        if len(hist) > 1:
            print(f"\nOrganization {hist[0]['id_']} has changed its name:")
            for row in hist:
                print(f"    {row['id_']} was named {row['name']} from {row['start_year']} to {row['end_year']}.")
    
    if max([len(find_name(name)) for name in names]) < 2:
        print('No organizations were renamed at any time.')

report_names(['4907', '3445'])


Organization 3445 has changed its name:
    3445 was named Dixie Pipeline from 2007 to 2010.
    3445 was named Dixie Pipeline Company from 2004 to 2006.
    3445 was named Dixie Pipeline Company Llc from 2011 to 2018.


In [9]:
def find_total_miles_year(OPERATOR_ID, year, df = pipelines_2010_raw, year_col = 'REPORT_YEAR', 
                          id_col = 'OPERATOR_ID', miles_col='PARTBHCATOTAL'):
    observations = df.loc[(df[year_col] == year) & (df[id_col] == int(OPERATOR_ID))][miles_col]
    return observations.sum()

find_total_miles_year('4906', 2015)

2113.75

In [10]:
def find_total_miles(OPERATOR_ID, years, df = pipelines_2010_raw, year_col = 'REPORT_YEAR', miles_col='PARTBHCATOTAL', id_col='OPERATOR_ID'):
    result = []
    for year in years:
        result.append([year, find_total_miles_year(OPERATOR_ID, year, df=df, year_col=year_col, id_col=id_col, miles_col=miles_col)])
    return(result)

find_total_miles('4906', range(2013, 2017))

[[2013, 2944.4], [2014, 2343.06], [2015, 2113.75], [2016, 2590.17]]

In [11]:
find_total_miles_2004 = partial(find_total_miles, df = pipelines_2004_raw, year_col = 'YR', miles_col = 'HCAONM')
find_total_miles_2004(31579, range(2007, 2010))

[[2007, 12.0], [2008, 14.0], [2009, 14.0]]

## 8.2 Iterate over issues and resolve them one at a time

We use the dictionary "company groups" to capture any companies we have to combine to form one observation. At every step we consult ownership_changes.txt to see if there are changed in ownership over time.

### Store results in

In [12]:
company_groups = pd.DataFrame()

def add_company_group(name: str, members: list, df = company_groups):
    new_group = pd.DataFrame({'members': members})
    new_group['name'] = name
    df = df.append(new_group, ignore_index=True)
    return df

In [13]:
m_as = pd.DataFrame()

def add_m_a(name: str, members: list, df, start_year: str = None, end_year: str = None):
    new_group = pd.DataFrame({'members': members})
    new_group['name'] = name
    
    if start_year:
        new_group['start_year'] = start_year
    if end_year:
        new_group['end_year'] = end_year
    
    df = df.append(new_group, sort=False, ignore_index=True)
    return df

In [14]:
spin_offs = []

### 8.2.1 Enterprise Products Operating

In [15]:
print(next(issues))


ENTERPRISE PRODUCTS OPERATING LLC (OPERATOR_ID 31618) has the same parent company as:

	DIXIE PIPELINE COMPANY LLC (OPERATOR_ID 3445)

	OILTANKING, HOUSTON LP (OPERATOR_ID 14194)

	ENTERPRISE CRUDE PIPELINE LLC (OPERATOR_ID 30829)

	TRI-STATES NGL PIPELINE LLC (OPERATOR_ID 31270)

	TEPPCO MIDSTREAM COMPANIES, LLC (OPERATOR_ID 32209)



In [16]:
report_names(['31618', '3445', '14194', '30829', '31270', '32209'])


Organization 3445 has changed its name:
    3445 was named Dixie Pipeline from 2007 to 2010.
    3445 was named Dixie Pipeline Company from 2004 to 2006.
    3445 was named Dixie Pipeline Company Llc from 2011 to 2018.

Organization 14194 has changed its name:
    14194 was named Oil Tanking Houston LP from 2005 to 2005.
    14194 was named Oiltanking Houston LP from 2006 to 2006.
    14194 was named Oiltanking Houston Lp from 2004 to 2018.

Organization 30829 has changed its name:
    30829 was named Enterprise Crude Pipeline Llc from 2010 to 2018.
    30829 was named Teppco Crude Pipeline LP from 2004 to 2006.
    30829 was named Teppco Crude Pipeline LP  from 2004 to 2004.
    30829 was named Teppco Crude Pipeline Llc from 2007 to 2009.
    30829 was named Tteppco Crude Pipeline LP from 2005 to 2005.


As per ownership_changes.txt, enterprise products hold a majority stake in Dixie Pipelines since at leat 2005, since data on an M&A is missing, we assume it has held a majority stake for the whole observation period before that. Teppco was acquired on Oct 26, 2009 (Deal Number 511604).

In [17]:
company_groups = add_company_group('Enterprise Products (Group)', ['31618', '14194', '31270'], company_groups)
company_groups = add_company_group('Teppco (Group)', ['30829', '32209', '3445'])
company_groups.tail()

,members,name
0,30829,Teppco (Group)
1,32209,Teppco (Group)
2,3445,Teppco (Group)


In [18]:
m_as = add_m_a('Enterprise Products (Group)', ['Enterprise Products (Group)', 'Teppco (Group)'], start_year = '2010')
m_as.tail()

,members,name,start_year
0,Enterprise Products (Group),Enterprise Products (Group),2010
1,Teppco (Group),Enterprise Products (Group),2010


### 8.2.2 Oneok NGL

In [19]:
print(next(issues))


ONEOK NGL PIPELINE, LLC (OPERATOR_ID 32109) has changed its name:

	Was named ONEOK NGL PIPELINE LP from 2005 to 2011.

	Was named ONEOK NGL PIPELINE, LLC from 2012 to 2018.

ONEOK NGL PIPELINE, LLC (OPERATOR_ID 32109) has the same parent company as:

	ONEOK FIELD SERVICES  (OPERATOR_ID 30629)

ONEOK NGL PIPELINE, LLC (OPERATOR_ID 32109) may have a namesake or namesakes:

	ONEOK, INC  (OPERATOR_ID 30629)



In [20]:
company_groups = add_company_group('ONEOK (Group)', ['32109', '30629'], company_groups)

### 8.2.3 Phillips 66

In [21]:
print(next(issues))


PHILLIPS 66 PIPELINE LLC (OPERATOR_ID 31684) has changed its name:

	Was named CONOCOPHILLIPS from 2007 to 2010.

	Was named CONOCOPHILLIPS - LOS ANGELES REFINERY from 2004 to 2011.

	Was named CONOCOPHILLIPS PIPE LINE CO. from 2004 to 2005.

	Was named CONOCOPHILLIPS PIPE LINE COMPANY from 2004 to 2006.

	Was named PHILLIPS 66 PIPELINE LLC from 2011 to 2018.

PHILLIPS 66 PIPELINE LLC (OPERATOR_ID 31684) has the same parent company as:

	PHILLIPS 66 COMPANY - SWEENY REFINERY (OPERATOR_ID 15485)

PHILLIPS 66 PIPELINE LLC (OPERATOR_ID 31684) may have a namesake or namesakes:

	CONOCOPHILLIPS COMPANY (OPERATOR_ID 15485)

	PHILLIPS 66 COMPANY (OPERATOR_ID 15485)



Downstream business was spun off in 2011 as Phillips 66.

Sweeny refinery is not an independent organizations. We will reconsolidate the two organizations.

### Decision

In [22]:
company_groups = add_company_group('Phillips 66 (Group)', ['15485', '31684'], company_groups)
company_groups.tail()

,members,name
2,3445,Teppco (Group)
3,32109,ONEOK (Group)
4,30629,ONEOK (Group)
5,15485,Phillips 66 (Group)
6,31684,Phillips 66 (Group)


### 8.2.4 Magellan

In [23]:
print(next(issues))


MAGELLAN PIPELINE COMPANY, LP (OPERATOR_ID 22610) has changed its name:

	Was named MAGELLAN PIPELINE COMPANY, L.P. from 2005 to 2006.

	Was named MAGELLAN PIPELINE COMPANY, LLC from 2004 to 2004.

	Was named MAGELLAN PIPELINE COMPANY, LP from 2007 to 2018.

	Was named Magellan Pipeline Company, L.P. from 2010 to 2010.

MAGELLAN PIPELINE COMPANY, LP (OPERATOR_ID 22610) has the same parent company as:

	MAGELLAN AMMONIA PIPELINE, L.P. (OPERATOR_ID 12105)

	MAGELLAN PIPELINES HOLDINGS, LP (OPERATOR_ID 31579)

	MAGELLAN CRUDE OIL PIPELINE COMPANY, L.P. (OPERATOR_ID 39504)

MAGELLAN PIPELINE COMPANY, LP (OPERATOR_ID 22610) may have a namesake or namesakes:

	Magellan Ammonia Pipeline, L.P. (OPERATOR_ID 12105)

	Magellan Pipelines Holdings, L.P. (OPERATOR_ID 31579)

	MAGELLAN PIPELINES HOLDINGS, L.P. (OPERATOR_ID 31579)



In [24]:
report_names(['22610', '12105', '31579', '39504'])


Organization 22610 has changed its name:
    22610 was named Magellan Pipeline Company LP from 2005 to 2010.
    22610 was named Magellan Pipeline Company Llc from 2004 to 2004.
    22610 was named Magellan Pipeline Company Lp from 2007 to 2018.


LexisNexis yielded no reports of ownership changed for any subsidiaries (some assets changed hands, but that case is appropriately covered by the FERC data).

### Decision

In [25]:
company_groups = add_company_group('Magellan (Group)', ['22610', '12105', '31579', '39504'], company_groups)
company_groups.tail()

,members,name
6,31684,Phillips 66 (Group)
7,22610,Magellan (Group)
8,12105,Magellan (Group)
9,31579,Magellan (Group)
10,39504,Magellan (Group)


### 8.2.5 Colonial Pipeline

In [26]:
print(next(issues))


COLONIAL PIPELINE CO (OPERATOR_ID 2552) has changed its name:

	Was named COLONIAL PIPELINE CO from 2007 to 2018.

	Was named COLONIAL PIPELINE COMPANY from 2004 to 2006.



Not a concern.

### 8.2.6 Buckeye

In [27]:
print(next(issues))


BUCKEYE PARTNERS, LP (OPERATOR_ID 1845) has the same parent company as:

	BUCKEYE DEVELOPMENT & LOGISTICS, LLC (OPERATOR_ID 31371)

BUCKEYE PARTNERS, LP (OPERATOR_ID 1845) may have a namesake or namesakes:

	 BUCKEYE PARTNERS, LP (OPERATOR_ID 31371)

	BUCKEYE PARTNERS, LP  (OPERATOR_ID 31371)

	BUCKEYE PARTNERS, LP (OPERATOR_ID 31371)

	BUCKEYE GULF COAST PIPELINE LP (OPERATOR_ID 31371)



### Decision

LexisNexis does not indicate that Buckeye Development & Logistics is an outside acquisition by Buckeye Partners. We consolidate the two organizations into one.

In [28]:
company_groups = add_company_group('Buckeye (Group)', ['1845', '31371'], company_groups)
company_groups.tail()

,members,name
8,12105,Magellan (Group)
9,31579,Magellan (Group)
10,39504,Magellan (Group)
11,1845,Buckeye (Group)
12,31371,Buckeye (Group)


### 8.2.7 Sunoco

In [29]:
print(next(issues))


SUNOCO PIPELINE L.P. (OPERATOR_ID 18718) has changed its name:

	Was named SUNOCO PIPELINE L.P. from 2007 to 2018.

	Was named SUNOCO PIPELINE L.P.  - WESTERN AREA from 2004 to 2006.

	Was named SUNOCO PIPELINE L.P. (EASTERN AREA) from 2004 to 2006.

	Was named SUNOCO PIPELINE L.P. - WESTERN AREA from 2005 to 2005.

SUNOCO PIPELINE L.P. (OPERATOR_ID 18718) has the same parent company as:

	HARBOR PIPELINE CO (OPERATOR_ID 7063)

	MID - VALLEY PIPELINE CO (OPERATOR_ID 12470)

	WEST TEXAS GULF PIPELINE CO (OPERATOR_ID 22442)

	ENERGY TRANSFER COMPANY (OPERATOR_ID 32099)

	INLAND CORPORATION (OPERATOR_ID 32683)

	DAPL-ETCO OPERATIONS MANAGEMENT, LLC (OPERATOR_ID 39205)

	PERMIAN EXPRESS PARTNERS LLC (OPERATOR_ID 39596)

SUNOCO PIPELINE L.P. (OPERATOR_ID 18718) may have a namesake or namesakes:

	HARBOR PIPELINE (OPERATED BY SUNOCO PIPELINE L.P.) (OPERATOR_ID 7063)

	OPERATED BY SUNOCO PIPELINE L.P. (OPERATOR_ID 7063)



In [30]:
report_names(['18718', '7063', '12470', '22442', '32099', '32683', '39205', '39596'])


Organization 18718 has changed its name:
    18718 was named Sunoco Pipeline LP from 2007 to 2018.
    18718 was named Sunoco Pipeline LP   Western Area from 2004 to 2006.
    18718 was named Sunoco Pipeline LP  Western Area from 2005 to 2005.
    18718 was named Sunoco Pipeline LP Eastern Area from 2004 to 2006.

Organization 7063 has changed its name:
    7063 was named Harbor Pipeline from 2006 to 2006.
    7063 was named Harbor Pipeline Co from 2007 to 2018.
    7063 was named Harbor Pipeline Operated By Sunoco Pipeline LP from 2004 to 2005.

Organization 12470 has changed its name:
    12470 was named Mid  Valley Pipeline Co from 2007 to 2018.
    12470 was named MidValley Pipeline Co from 2005 to 2006.
    12470 was named MidValley Pipeline Company from 2004 to 2004.

Organization 22442 has changed its name:
    22442 was named West Texas Gulf Pipeline Co from 2005 to 2018.
    22442 was named West Texas Gulf Pipeline Company from 2004 to 2004.


According to our .txt file on ownership changes, Sunoco 

    - acquired a majority stake in Inland Corp (32683) on May 17, 2011
    
    - acquired a majority stake in West Texas Gulf Pipeline (22442) on Aug 18, 2010
    
    - merged with Energy Transfer (32099) on Oct 5, 2012

In [31]:
company_groups = add_company_group('Sunoco (Group)', ['18718', '12470', '39205', '39596', '7063'], company_groups)
company_groups = add_company_group('Energy Transfer Partners (Group)', ['32099'], company_groups)
company_groups.tail()

,members,name
14,12470,Sunoco (Group)
15,39205,Sunoco (Group)
16,39596,Sunoco (Group)
17,7063,Sunoco (Group)
18,32099,Energy Transfer Partners (Group)


In [32]:
m_as = add_m_a('Sunoco (Group)', ['Sunoco (Group)', '32683', '22442'], m_as, start_year = '2011')
m_as = add_m_a('Sunoco (Group)', ['Sunoco (Group)', 'Energy Transfer Partners (Group)'], m_as, start_year = '2013')

m_as.tail()

,members,name,start_year
2,Sunoco (Group),Sunoco (Group),2011
3,32683,Sunoco (Group),2011
4,22442,Sunoco (Group),2011
5,Sunoco (Group),Sunoco (Group),2013
6,Energy Transfer Partners (Group),Sunoco (Group),2013


### 8.2.8 Flint Hills

In [33]:
print(next(issues))


FLINT HILLS RESOURCES, LC (OPERATOR_ID 22855) has changed its name:

	Was named  KOCH PIPELINE COMPANY, LP from 2004 to 2004.

	Was named FLINT HILLS RESOURCES, LC from 2017 to 2018.

	Was named KOCH PIPELINE COMPANY, L.P. from 2007 to 2016.

	Was named KOCH PIPELINE COMPANY, LP from 2004 to 2006.

	Was named Koch Pipeline Company, LP from 2005 to 2005.



In [34]:
find_total_miles('22855', range(2015, 2019))

[[2015, 1885.19], [2016, 1674.88], [2017, 1711.48], [2018, 1750.7199999999998]]

The restructuring or sell-off does not seem to affect the organizations pipeline assets. We will ignore the rebranding efforts.

### 3.2.9 Kinder Morgan

In [35]:
print(next(issues))


TE PRODUCTS PIPELINE COMPANY, LLC (OPERATOR_ID 19237) has changed its name:

	Was named TE PRODUCTS PIPELINE CO., LP  from 2004 to 2004.

	Was named TE PRODUCTS PIPELINE COMPANY, LIMITED PARTNERSHIP from 2005 to 2005.

	Was named TE PRODUCTS PIPELINE COMPANY, LLC from 2007 to 2009.

	Was named TE PRODUCTS PIPELINE, LIMITED PARTNERSHIP from 2006 to 2006.

TE PRODUCTS PIPELINE COMPANY, LLC (OPERATOR_ID 19237) has the same parent company as:

	CENTRAL FLORIDA PIPELINE CORP (OPERATOR_ID 2190)

	CYPRESS INTERSTATE PIPELINE LLC (OPERATOR_ID 4472)

	PLANTATION PIPE LINE CO (OPERATOR_ID 15674)

	SFPP, LP (OPERATOR_ID 18092)

	TRANS MOUNTAIN PIPELINE (PUGET SOUND) LLC (OPERATOR_ID 19585)

	CALNEV PIPELINE CO (OPERATOR_ID 26125)

	KINDER MORGAN WINK PIPELINE LLC (OPERATOR_ID 31957)

	COPANO NGL SERVICES LLC (OPERATOR_ID 32114)

	KINDER MORGAN COCHIN LLC (OPERATOR_ID 32258)

	COPANO NGL SERVICES (MARKHAM), LLC (OPERATOR_ID 32541)

	HILAND CRUDE, LLC (OPERATOR_ID 32619)

	KINDER MORGAN CRUDE AND 

In [36]:
report_names(['19237', '2190', '4472', '15674', '18092', '19585', '26125', '31555', '31957', '32114', 
              '32258', '32541', '32619', '32678', '39023', '39440', '39518'])


Organization 19237 has changed its name:
    19237 was named Te Products Pipeline Co Lp  from 2004 to 2004.
    19237 was named Te Products Pipeline Company Limited Partnership from 2005 to 2005.
    19237 was named Te Products Pipeline Company Llc from 2007 to 2009.
    19237 was named Te Products Pipeline Limited Partnership from 2006 to 2006.

Organization 2190 has changed its name:
    2190 was named Central Florida Pipeline Corp from 2007 to 2018.
    2190 was named Central Florida Pipeline Llc from 2004 to 2006.

Organization 4472 has changed its name:
    4472 was named Cypress Interstate Pipeline Llc from 2012 to 2018.
    4472 was named Kinder Morgan Energy Partners LP from 2004 to 2011.

Organization 15674 has changed its name:
    15674 was named Plantation Pipe Line Co from 2007 to 2018.
    15674 was named Plantation Pipe Line Company from 2004 to 2006.

Organization 19585 has changed its name:
    19585 was named Terasen Pipeline Puget Sound Corp from 2007 to 2007.
    1

According to our .txt file on ownership changes, Kinder Morgan:

    - has held a majority share in Plantation Pipe Line (15674) since June 16, 1999
    
    - sold off Cochin (32258) to Pembina on Aug 21, 2019

In [37]:
company_groups = add_company_group('Kinder Morgan (Group)', 
                                   ['19237', '2190', '4472', '15674', '18092', '19585', '26125', '31555', '31957', '32114', 
                                    '32258', '32541', '32619', '32678', '39023', '39440', '39518'], 
                                   company_groups)
company_groups.tail()

,members,name
31,32619,Kinder Morgan (Group)
32,32678,Kinder Morgan (Group)
33,39023,Kinder Morgan (Group)
34,39440,Kinder Morgan (Group)
35,39518,Kinder Morgan (Group)


### 8.2.10 NuStar

In [38]:
print(next(issues))


NUSTAR PIPELINE OPERATING PARTNERSHIP L.P. (OPERATOR_ID 10012) has changed its name:

	Was named KANEB PIPE LINE COMPANY LLC from 2004 to 2005.

	Was named KANEB PIPE LINE OPERATING PARTNERSHIP, L.P. from 2006 to 2006.

	Was named NUSTAR PIPELINE OPERATING PARTNERSHIP L.P. from 2007 to 2018.

NUSTAR PIPELINE OPERATING PARTNERSHIP L.P. (OPERATOR_ID 10012) has the same parent company as:

	NUSTAR TERMINALS OPERATIONS PARTNERSHIP L. P. (OPERATOR_ID 26094)

	NUSTAR LOGISTICS, L.P. (OPERATOR_ID 31454)

	NUSTAR PERMIAN TRANSPORTATION AND STORAGE, LLC (OPERATOR_ID 39348)

NUSTAR PIPELINE OPERATING PARTNERSHIP L.P. (OPERATOR_ID 10012) may have a namesake or namesakes:

	NUSTAR LOGISITICS, L.P. (OPERATOR_ID 31454)



In [39]:
report_names(['10012', '26094', '31454', '39348'])


Organization 10012 has changed its name:
    10012 was named Kaneb Pipe Line Company Llc from 2004 to 2005.
    10012 was named Kaneb Pipe Line Operating Partnership LP from 2006 to 2006.
    10012 was named Nustar Pipeline Operating Partnership LP from 2007 to 2018.

Organization 26094 has changed its name:
    26094 was named Nustar Terminals Operations Partnership L P from 2007 to 2018.
    26094 was named Support Terminals Operating Partnership from 2004 to 2004.
    26094 was named Support Terminals Operating Partnership LP from 2005 to 2006.

Organization 31454 has changed its name:
    31454 was named Nustar Logisitics LP from 2006 to 2006.
    31454 was named Nustar Logistics LP from 2007 to 2018.
    31454 was named Valero Logistics Operations Lp from 2004 to 2005.
    31454 was named Valero Logistics Operations Lp  from 2004 to 2004.


Before Kaneb became Nustar Pipeline, it was operated by a partnership between Valero and NuStar (http://nustarenergy.com/en-us/Company/Pages/History.aspx).

LexisNexis Doessier does not list any M&As for the other three subsidiaries. They just seem to be two separate subsidiaries, that we can group together.

In [40]:
company_groups = add_company_group('NuStar (Group)', ['26094', '31454', '39348'], company_groups)
company_groups.tail()

,members,name
34,39440,Kinder Morgan (Group)
35,39518,Kinder Morgan (Group)
36,26094,NuStar (Group)
37,31454,NuStar (Group)
38,39348,NuStar (Group)


In [41]:
m_as = add_m_a('NuStar (Group)', ['NuStar (Group)', '10012'], m_as, start_year = '2007')
m_as = add_m_a('NuStar (Group)', ['NuStar (Group)', '31454'], m_as, start_year = '2006')
m_as.tail()

,members,name,start_year
6,Energy Transfer Partners (Group),Sunoco (Group),2013
7,NuStar (Group),NuStar (Group),2007
8,10012,NuStar (Group),2007
9,NuStar (Group),NuStar (Group),2006
10,31454,NuStar (Group),2006


### 8.2.11 Plantation Pipeline

In [42]:
print(next(issues))


PLANTATION PIPE LINE CO (OPERATOR_ID 15674) has changed its name:

	Was named PLANTATION PIPE LINE CO from 2007 to 2018.

	Was named PLANTATION PIPE LINE COMPANY from 2004 to 2006.



Not a concern.

### 8.2.12 Enbridge

In [43]:
print(next(issues))


ENBRIDGE ENERGY, LIMITED PARTNERSHIP (OPERATOR_ID 11169) has the same parent company as:

	ENBRIDGE PIPELINES (TOLEDO) INC (OPERATOR_ID 31448)

	EXPRESS HOLDINGS (USA), LLC (OPERATOR_ID 31720)

	ENBRIDGE PIPELINES (OZARK) L.L.C. (OPERATOR_ID 31947)

	CCPS TRANSPORTATION, LLC (OPERATOR_ID 32080)

	ENBRIDGE PIPELINES (SOUTHERN LIGHTS) L.L.C. (OPERATOR_ID 32502)

ENBRIDGE ENERGY, LIMITED PARTNERSHIP (OPERATOR_ID 11169) may have a namesake or namesakes:

	ENBRIDGE PIPELINES (NORTH DAKOTA) LLC (OPERATOR_ID 15774)

	ENBRIDGE ENERGY PARTNERS, LP (OPERATOR_ID 15774)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 15774)

	ENBRIDGE (US) INC (OPERATOR_ID 31448)

	ENBRIDGE (U.S.) INC. (OPERATOR_ID 31448)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 31448)

	ENBRIDGE ENERGY PARTNERS, LP (OPERATOR_ID 31947)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 31947)

	ENBRIDGE ENERGY COMPANY INC. (OPERATOR_ID 32080)

	ENBRIDGE ENERGY COMPANY, INC. (OPERATOR_ID 32080)



In [44]:
report_names(['11169', '31448', '31720', '31947', '32080', '15774', '32502'])


Organization 31720 has changed its name:
    31720 was named Express Holdings Usa Llc from 2013 to 2018.
    31720 was named Kinder Morgan Pipelines Usa Inc from 2008 to 2012.
    31720 was named Terasen Pipeline Usa Inc  from 2004 to 2004.
    31720 was named Terasen Pipelines Usa Inc from 2005 to 2007.

Organization 15774 has changed its name:
    15774 was named Enbridge Pipelines North Dakota Llc from 2004 to 2012.
    15774 was named North Dakota Pipeline Company Llc from 2013 to 2018.


Spectra Energy bought Express Pipeline (31720) in 2012 from a number of stakeholders, including Kinder Morgan (Deal Number 741207). Enbridge merged with Spectra Energy on Feb 27, 2017 (LexisNexis Dossier Deal Number 885812).

North Dakota Pipeline Company is a subsidiary of Marathon, so we will assume that this asset has been bough by marathon in 2013.

In [45]:
company_groups = add_company_group('Enbridge (Group)', 
                                   ['11169', '31448', '31720', '31947', '32080', '32502'], company_groups)
company_groups.tail()

,members,name
41,31720,Enbridge (Group)
42,31947,Enbridge (Group)
43,32080,Enbridge (Group)
44,31947,Enbridge (Group)
45,32502,Enbridge (Group)


In [46]:
m_as = add_m_a('Enbridge (Group)', ['Enbridge (Group)', '15774'], m_as, end_year='2013')
m_as = add_m_a('Enbridge (Group)', ['Enbridge (Group)', '31720'], m_as, start_year='2012')
m_as.tail()

,members,name,start_year,end_year
10,31454,NuStar (Group),2006,NaN
11,Enbridge (Group),Enbridge (Group),NaN,2013
12,15774,Enbridge (Group),NaN,2013
13,Enbridge (Group),Enbridge (Group),2012,NaN
14,31720,Enbridge (Group),2012,NaN


According to our notes, Enbridge also acquired a majority in Olympic pipeline (30781) from BP on Feb 1, 2006.

In [47]:
m_as = add_m_a('Enbridge (Group)', ['Enbridge (Group)', '30781'], m_as, start_year='2006')
m_as = add_m_a('BP (Group)', ['BP (Group)', '30781'], m_as, start_year='2001', end_year='2006')
m_as.tail()

,members,name,start_year,end_year
14,31720,Enbridge (Group),2012,NaN
15,Enbridge (Group),Enbridge (Group),2006,NaN
16,30781,Enbridge (Group),2006,NaN
17,BP (Group),BP (Group),2001,2006
18,30781,BP (Group),2001,2006


### 9.2.13 Marathon

In [48]:
print(next(issues))


MARATHON PIPE LINE LLC (OPERATOR_ID 32147) has changed its name:

	Was named MARATHON ASHLAND PIPE LINE, LLC from 2005 to 2005.

	Was named MARATHON PIPE LINE LLC from 2005 to 2018.

MARATHON PIPE LINE LLC (OPERATOR_ID 32147) has the same parent company as:

	NORTH DAKOTA PIPELINE COMPANY LLC (OPERATOR_ID 15774)

	WOLVERINE PIPELINE CO (OPERATOR_ID 22830)

	MARKWEST RANGER PIPELINE COMPANY, L.L.C. (OPERATOR_ID 26026)

	TESORO HIGH PLAINS PIPELINE COMPANY LLC (OPERATOR_ID 31570)

	WESTERN REFINING LOGISTICS, LP (OPERATOR_ID 31574)

	TESORO LOGISTICS OPERATIONS LLC - MOUNTAIN REGION (OPERATOR_ID 31583)

	MARKWEST MICHIGAN PIPELINE COMPANY, L.L.C. (OPERATOR_ID 31871)

	TESORO LOGISTICS OPERATIONS LLC - GOLDEN EAGLE (OPERATOR_ID 31874)

	TESORO LOGISTICS OPERATIONS, LLC (OPERATOR_ID 38933)

	TESORO SOCAL PIPELINE COMPANY LLC (OPERATOR_ID 39013)

	TESORO LOGISTICS NORTHWEST PIPELINE LLC (OPERATOR_ID 39029)

	ILLINOIS EXTENSION PIPELINE COMPANY, L.L.C. (OPERATOR_ID 39347)

	MARATHON ASHLAND

In [49]:
report_names(['32147', '15774', '22830', '26026', '31570', '31574', '31583', '31871', '38933', '39013', '39029', '39347', '12127'])


Organization 32147 has changed its name:
    32147 was named Marathon Ashland Pipe Line Llc from 2005 to 2005.
    32147 was named Marathon Pipe Line Llc from 2005 to 2018.

Organization 15774 has changed its name:
    15774 was named Enbridge Pipelines North Dakota Llc from 2004 to 2012.
    15774 was named North Dakota Pipeline Company Llc from 2013 to 2018.

Organization 26026 has changed its name:
    26026 was named Markwest Energy Appalachia LLC from 2005 to 2006.
    26026 was named Markwest Energy Appalachia Llc from 2004 to 2010.
    26026 was named Markwest Ranger Pipeline Company LLC from 2012 to 2018.
    26026 was named Markwest Ranger Pipeline Company Llc from 2011 to 2011.

Organization 31570 has changed its name:
    31570 was named Tesoro  High Plains Pipeline Company from 2007 to 2011.
    31570 was named Tesoro High Plain Pipeline Company from 2006 to 2006.
    31570 was named Tesoro High Plains Pipeline Company from 2005 to 2005.
    31570 was named Tesoro High Pla

As we have touched on above, the North Dakota pipeline's owner was named "Enbridge Pipelines", but it was actually owned by a consortium of different actors.

LexisNexis indicates that Tesoro was renamed to Andeavor, and acquired by Marathon on Oct 1, 2018 (LexisNexis Dossier Deal Number 3052392). We will group Wolverine with Marathon, and also combine all Tesoros into one observation. For 2019, we would subsume Tesoro under Marathon.

In [50]:
# Marathon and Wolverine
company_groups = add_company_group('Marathon (Group)', 
                                   ['32147', '22830', '26026', '31574', '31871', '39347', '12127'], company_groups)

# Tesoro
company_groups = add_company_group('Tesoro (Group)', ['31570', '31583', '38933', '39013', '39029'], company_groups)
company_groups.tail()

,members,name
53,31570,Tesoro (Group)
54,31583,Tesoro (Group)
55,38933,Tesoro (Group)
56,39013,Tesoro (Group)
57,39029,Tesoro (Group)


North Dakota Pipeline was acquired from Enbridge in 2013.

In [51]:
m_as = add_m_a('Marathon (Group)', ['Marathon (Group)', '15774'], m_as, start_year='2013')

### 8.2.14 ExxonMobil

In [52]:
print(next(issues))


EXXONMOBIL PIPELINE CO (OPERATOR_ID 4906) has changed its name:

	Was named EXXONMOBIL PIPELINE CO from 2007 to 2018.

	Was named EXXONMOBIL PIPELINE COMPANY from 2004 to 2006.

EXXONMOBIL PIPELINE CO (OPERATOR_ID 4906) has the same parent company as:

	EXXONMOBIL OIL CORPORATION (OPERATOR_ID 12624)

	EXXONMOBIL PIPELINE CO (OPERATOR_ID 12628)

	EXXONMOBIL OIL CORPORATION (OPERATOR_ID 12634)

	EXXONMOBIL OIL CORP - WEST COAST (OPERATOR_ID 26134)

	MOBIL PACIFIC PIPELINE CO (OPERATOR_ID 30005)

EXXONMOBIL PIPELINE CO (OPERATOR_ID 4906) may have a namesake or namesakes:

	MOBIL CORP (OPERATOR_ID 12624)

	MOBIL OIL CORPORATION - MOBIL BEAUMNT REFINERY (HULL) (OPERATOR_ID 12624)

	MOBIL OIL  CORPORATION-MOBIL BEAUMONT REFINERY (MAGPETCO) (OPERATOR_ID 12624)

	MOBIL OIL CORPORATION - MOBIL BEAUMONT REFINERY (HULL) (OPERATOR_ID 12624)

	EXXONMOBIL OIL CORPORATION - BEAUMONT REFINERY (HULL) (OPERATOR_ID 12624)

	EXXONMOBIL OIL CORPORATION - BEAUMONT REFINERY (MAGPETCO) (OPERATOR_ID 12624)

	

In [53]:
report_names(['4906', '12624', '12628', '12634', '30005'])


Organization 4906 has changed its name:
    4906 was named Exxonmobil Pipeline Co from 2007 to 2018.
    4906 was named Exxonmobil Pipeline Company from 2004 to 2006.

Organization 12624 has changed its name:
    12624 was named Exxonmobil Oil Corporation from 2018 to 2018.
    12624 was named Exxonmobil Oil Corporation  Beaumont Refinery Hull from 2006 to 2006.
    12624 was named Exxonmobil Oil Corporation  Beaumont Refinery Magpetco from 2006 to 2006.
    12624 was named Mobil Corp from 2004 to 2017.
    12624 was named Mobil Oil  CorporationMobil Beaumont Refinery Magpetco from 2004 to 2004.
    12624 was named Mobil Oil Corporation  Mobil Beaumnt Refinery Hull from 2004 to 2004.
    12624 was named Mobil Oil Corporation  Mobil Beaumont Refinery Hull from 2005 to 2005.
    12624 was named Mobil Oil Corporation  Mobil Beaumont Refinery Magpetco from 2005 to 2005.

Organization 12628 has changed its name:
    12628 was named Exxonmobil Pipeline Co from 2007 to 2018.
    12628 was na

All are wholly owned subsidiaries of exxonmobil. LexisNexis indicates no M&A activities. We combine them into one observation.

In [54]:
company_groups = add_company_group('ExxonMobil (Group)', ['4906', '12624', '12628', '12634', '30005'], company_groups)
company_groups.tail()

,members,name
58,4906,ExxonMobil (Group)
59,12624,ExxonMobil (Group)
60,12628,ExxonMobil (Group)
61,12634,ExxonMobil (Group)
62,30005,ExxonMobil (Group)


### 8.2.15 Chevron

In [55]:
print(next(issues))


CHEVRON PIPE LINE CO (OPERATOR_ID 2731) has changed its name:

	Was named CHEVRON PIPE LINE CO from 2007 to 2018.

	Was named CHEVRON PIPE LINE COMPANY from 2004 to 2006.

	Was named CHEVRON PIPELINE COMPANY from 2004 to 2004.

CHEVRON PIPE LINE CO (OPERATOR_ID 2731) has the same parent company as:

	CHEVRON PRODUCTS COMPANY - HAWAII (OPERATOR_ID 2339)

	CHEVRON MIDSTREAM PIPELINES LLC (OPERATOR_ID 31556)

CHEVRON PIPE LINE CO (OPERATOR_ID 2731) may have a namesake or namesakes:

	CHEVRON PETROCHEMICAL PIPELINE LLC (OPERATOR_ID 31554)

	CHEVRON PETROCHEMICAL PIPELINE LLC (FORMERLY TEXACO PETROCHEMICAL PIPELINE LLC) (OPERATOR_ID 31554)

	CHEVRON USA, INC. (OPERATOR_ID 2339)

	CHEVRON USA,  INC. (OPERATOR_ID 2339)

	CHEVRON PRODUCTS COMPANY - HAWAII  (OPERATOR_ID 2339)



In [56]:
report_names(['2731', '2339', '31556', '31554'])


Organization 2731 has changed its name:
    2731 was named Chevron Pipe Line Co from 2007 to 2018.
    2731 was named Chevron Pipe Line Company from 2004 to 2006.
    2731 was named Chevron Pipeline Company from 2004 to 2004.

Organization 2339 has changed its name:
    2339 was named Chevron Products Company  Hawaii from 2010 to 2015.
    2339 was named Chevron Products Company  Hawaii  from 2007 to 2009.
    2339 was named Chevron Products Company Hawaii Refinery from 2004 to 2006.

Organization 31556 has changed its name:
    31556 was named Chevron Midstream Pipelines Llc from 2009 to 2015.
    31556 was named Texaco Pipelines Llc from 2004 to 2008.

Organization 31554 has changed its name:
    31554 was named Boardwalk Petrochemical Pipeline Llc from 2014 to 2018.
    31554 was named Chevron Petrochemical Pipeline Llc from 2007 to 2013.
    31554 was named Chevron Petrochemical Pipeline Llc Formerly Texaco Petrochemical Pipeline Llc from 2006 to 2006.
    31554 was named Texaco P

Texaco merged with Chevron in early 2002. Our notes indicate that Boardwalk is a subsidiary of the Loews Corporation (which does not hold any other pipeline assets) that acquired assets from Chevron on Oct 8, 2014.

In [57]:
company_groups = add_company_group('Chevron (Group)', ['2731', '2339'], company_groups)
company_groups.tail()

,members,name
60,12628,ExxonMobil (Group)
61,12634,ExxonMobil (Group)
62,30005,ExxonMobil (Group)
63,2731,Chevron (Group)
64,2339,Chevron (Group)


In [58]:
m_as = add_m_a('Chevron (Group)', ['Chevron (Group)', '31556'], m_as, start_year='2002')
m_as = add_m_a('Chevron (Group)', ['Chevron (Group)', '31554'], m_as, start_year='2002', end_year='2014')
m_as.tail()

,members,name,start_year,end_year
20,15774,Marathon (Group),2013,NaN
21,Chevron (Group),Chevron (Group),2002,NaN
22,31556,Chevron (Group),2002,NaN
23,Chevron (Group),Chevron (Group),2002,2014
24,31554,Chevron (Group),2002,2014


### 8.2.16 NuStar (repeat)

In [59]:
print(next(issues))


NUSTAR LOGISTICS, L.P. (OPERATOR_ID 31454) has changed its name:

	Was named NUSTAR LOGISITICS, L.P. from 2006 to 2006.

	Was named NUSTAR LOGISTICS, L.P. from 2007 to 2018.

	Was named VALERO LOGISTICS OPERATIONS, LP from 2004 to 2005.

	Was named VALERO LOGISTICS OPERATIONS, LP  from 2004 to 2004.

NUSTAR LOGISTICS, L.P. (OPERATOR_ID 31454) may have a namesake or namesakes:

	VALERO TERMINALING AND DISTRIBUTION COMPANY (OPERATOR_ID 4430)

	VALERO ENERGY CORPORATION (OPERATOR_ID 4430)

	VALERO PARTNERS OPERATING CO. LLC (OPERATOR_ID 39105)

	VALERO ENERGY CORPORATION (OPERATOR_ID 31742)

	VALERO REFINING - TEXAS LP (OPERATOR_ID 32032)



We have addressed NuStar above and will address Valero below.

### 8.2.17 SFPP

In [60]:
print(next(issues))


SFPP, LP (OPERATOR_ID 18092) has changed its name:

	Was named SFPP, L.P. from 2006 to 2006.

	Was named SFPP, LP from 2004 to 2018.



Not a concern.

### 8.2.18 Marathon (repeat)

In [61]:
print(next(issues))


MARATHON ASHLAND PIPE LINE LLC  (OPERATOR_ID 12127) may have a namesake or namesakes:

	MARATHON PETROLEUM COMPANY LLC (OPERATOR_ID 32147)



Already addressed above, not a concern.

### 8.2.19 Explorer Pipeline

In [62]:
print(next(issues))


EXPLORER PIPELINE CO (OPERATOR_ID 4805) has changed its name:

	Was named EXPLORER PIPELINE CO from 2007 to 2018.

	Was named EXPLORER PIPELINE COMPANY from 2004 to 2006.



Not a concern.

### 8.2.20 Buckeye

In [63]:
print(next(issues))


BUCKEYE DEVELOPMENT & LOGISTICS, LLC (OPERATOR_ID 31371) has changed its name:

	Was named BUCKEYE DEVELOPMENT & LOGISTICS, LLC from 2010 to 2018.

	Was named BUCKEYE GULF COAST PIPE LINES, LP from 2004 to 2006.

	Was named BUCKEYE GULF COAST PIPE LINES, LP  from 2004 to 2004.

	Was named BUCKEYE GULF COAST PIPELINE LP from 2007 to 2009.



Not a concern.

### 8.2.21 Enterprise

In [64]:
print(next(issues))


ENTERPRISE CRUDE PIPELINE LLC (OPERATOR_ID 30829) has changed its name:

	Was named ENTERPRISE CRUDE PIPELINE LLC from 2010 to 2018.

	Was named TEPPCO CRUDE PIPELINE, L.P. from 2004 to 2006.

	Was named TEPPCO CRUDE PIPELINE, L.P.  from 2004 to 2004.

	Was named TEPPCO CRUDE PIPELINE, LLC from 2007 to 2009.

	Was named TTEPPCO CRUDE PIPELINE, L.P. from 2005 to 2005.

ENTERPRISE CRUDE PIPELINE LLC (OPERATOR_ID 30829) may have a namesake or namesakes:

	TEPPCO MIDSTREAM COMPANIES, L.P. (OPERATOR_ID 32209)



Already addressed above.

### 8.2.22 Equistar

In [65]:
print(next(issues))


EQUISTAR CHEMICALS, L.P. (OPERATOR_ID 25146) has changed its name:

	Was named EQUISTAR CHEMICALS LP from 2004 to 2004.

	Was named EQUISTAR CHEMICALS, L.P. from 2007 to 2018.

	Was named EQUISTAR CHEMICALS, LP from 2005 to 2006.



Not a concern.

### 8.2.23 Teppco

In [66]:
print(next(issues))


TEPPCO MIDSTREAM COMPANIES, LLC (OPERATOR_ID 32209) has changed its name:

	Was named TEPPCO MIDSTREAM COMPANIES, L.P. from 2007 to 2008.

	Was named TEPPCO MIDSTREAM COMPANIES, LLC from 2009 to 2009.

TEPPCO MIDSTREAM COMPANIES, LLC (OPERATOR_ID 32209) may have a namesake or namesakes:

	TEPPCO CRUDE PIPELINE, L.P. (OPERATOR_ID 30829)

	TEPPCO CRUDE PIPELINE, L.P.  (OPERATOR_ID 30829)

	TTEPPCO CRUDE PIPELINE, L.P. (OPERATOR_ID 30829)

	TEPPCO CRUDE PIPELINE, LLC (OPERATOR_ID 30829)



Not a conern/already adressed above.

### 8.2.24 Kinder Morgan

In [67]:
print(next(issues))


KINDER MORGAN COCHIN LLC (OPERATOR_ID 32258) may have a namesake or namesakes:

	KINDER MORGAN ENERGY PARTNERS, L.P. (OPERATOR_ID 4472)

	KINDER MORGAN CANADA INC. (OPERATOR_ID 19585)

	KINDER MORGAN CANAADA INC. (OPERATOR_ID 19585)

	KINDER MORGAN CANADA INC.  (OPERATOR_ID 19585)

	KINDER MORGAN LIQUID TERMINALS, LLC (OPERATOR_ID 26041)

	KINDER MORGAN, INC (OPERATOR_ID 26041)

	KINDER MORGAN LIQUID TERMINAL, LLC (WEST COAST TERMINALS) (OPERATOR_ID 26041)

	KINDER MORGAN LIQUIDS TERMINALS LLC (OPERATOR_ID 26041)

	KINDER MORGAN LIQUIDS TERMINALS, L.P. (OPERATOR_ID 26041)

	KINDER MORGAN PIPELINES (USA) INC (OPERATOR_ID 31720)

	KINDER MORGAN CANADA INC. (OPERATOR_ID 31720)

	KINDER MORGAN CANADA, INC. (OPERATOR_ID 31720)

	KINDER MORGAN CANADA INC.  (OPERATOR_ID 31720)

	KINDER MORGAN WINK PIPELINE LP (OPERATOR_ID 31957)

	KINDER MORGAN WINK PIPELINE, LP (OPERATOR_ID 31957)



Not a concern/already addressed above.

### 8.2.25 Permian

In [68]:
print(next(issues))


PERMIAN EXPRESS PARTNERS LLC (OPERATOR_ID 39596) may have a namesake or namesakes:

	NUSTAR PERMIAN TRANSPORTATION AND STORAGE, LLC (OPERATOR_ID 39348)



Permian is already integrated with NuStar.

### 8.2.26 Tesoro (repeat)

In [69]:
print(next(issues))


TESORO LOGISTICS OPERATIONS, LLC (OPERATOR_ID 38933) has changed its name:

	Was named TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA from 2012 to 2017.

	Was named TESORO LOGISTICS OPERATIONS, LLC from 2018 to 2018.

TESORO LOGISTICS OPERATIONS, LLC (OPERATOR_ID 38933) may have a namesake or namesakes:

	TESORO - HIGH PLAINS PIPELINE COMPANY (OPERATOR_ID 31570)

	TESORO HIGH PLAINS PIPELINE COMPANY  (OPERATOR_ID 31570)

	TESORO CORPORATION (OPERATOR_ID 31570)

	TESORO REFINING & MARKETING CO., MOUNTAIN REGION (OPERATOR_ID 31583)

	TESORO REFINING & MARKETING CO, MOUNTAIN REGION  (OPERATOR_ID 31583)

	TESORO CORPORATION (OPERATOR_ID 31583)

	TESORO REFINING & MARKETING C O., GOLDEN EAGLE (OPERATOR_ID 31874)

	TESORO CORPORATION (OPERATOR_ID 31874)



Already covered above.

### 8.2.27 Plains Pipeline

In [70]:
print(next(issues))


ROCKY MOUNTAIN PIPELINE SYSTEM, LLC (OPERATOR_ID 31666) has changed its name:

	Was named ROCKY MOUNTAIN PIPELINE SYSTEM  LLC from 2006 to 2006.

	Was named ROCKY MOUNTAIN PIPELINE SYSTEM LLC from 2005 to 2005.

	Was named ROCKY MOUNTAIN PIPELINE SYSTEM, LLC from 2004 to 2010.

ROCKY MOUNTAIN PIPELINE SYSTEM, LLC (OPERATOR_ID 31666) has the same parent company as:

	PLAINS PIPELINE, L.P. (OPERATOR_ID 300)

	PLAINS MARKETING, L.P. (OPERATOR_ID 26085)



In [71]:
report_names(['31666', '300', '26085'])

No organizations were renamed at any time.


LexisNexis Doassiers indicates no M&As for the subsidiaries, Plains Marketing and Rocky Mountain Pipeline System.

In [72]:
company_groups = add_company_group('Plains Pipeline (Group)', ['300', '31666', '26085'], company_groups)
company_groups.tail()

,members,name
63,2731,Chevron (Group)
64,2339,Chevron (Group)
65,300,Plains Pipeline (Group)
66,31666,Plains Pipeline (Group)
67,26085,Plains Pipeline (Group)


### 3.2.28 Mid-Valley

In [73]:
print(next(issues))


MID - VALLEY PIPELINE CO (OPERATOR_ID 12470) has changed its name:

	Was named MID - VALLEY PIPELINE CO from 2007 to 2018.

	Was named MID-VALLEY PIPELINE CO from 2005 to 2005.

	Was named MID-VALLEY PIPELINE CO. from 2006 to 2006.

	Was named MID-VALLEY PIPELINE COMPANY from 2004 to 2004.



No concern.

### 8.2.29 Kinder Morgan (repeat)

In [74]:
print(next(issues))


CYPRESS INTERSTATE PIPELINE LLC (OPERATOR_ID 4472) has changed its name:

	Was named CYPRESS INTERSTATE PIPELINE LLC from 2012 to 2018.

	Was named KINDER MORGAN ENERGY PARTNERS, L.P. from 2004 to 2011.

CYPRESS INTERSTATE PIPELINE LLC (OPERATOR_ID 4472) may have a namesake or namesakes:

	KINDER MORGAN CANADA INC. (OPERATOR_ID 19585)

	KINDER MORGAN CANAADA INC. (OPERATOR_ID 19585)

	KINDER MORGAN CANADA INC.  (OPERATOR_ID 19585)

	KINDER MORGAN LIQUID TERMINALS, LLC (OPERATOR_ID 26041)

	KINDER MORGAN, INC (OPERATOR_ID 26041)

	KINDER MORGAN LIQUID TERMINAL, LLC (WEST COAST TERMINALS) (OPERATOR_ID 26041)

	KINDER MORGAN LIQUIDS TERMINALS LLC (OPERATOR_ID 26041)

	KINDER MORGAN LIQUIDS TERMINALS, L.P. (OPERATOR_ID 26041)

	KINDER MORGAN PIPELINES (USA) INC (OPERATOR_ID 31720)

	KINDER MORGAN CANADA INC. (OPERATOR_ID 31720)

	KINDER MORGAN CANADA, INC. (OPERATOR_ID 31720)

	KINDER MORGAN CANADA INC.  (OPERATOR_ID 31720)

	KINDER MORGAN WINK PIPELINE LP (OPERATOR_ID 31957)

	KINDER MOR

Already covered above.

### 8.2.30 Shell pipeline

In [75]:
print(next(issues))


SHELL PIPELINE CO., L.P. (OPERATOR_ID 31174) has changed its name:

	Was named SHELL PIPELINE CO., L.P. from 2007 to 2018.

	Was named SHELL PIPELINE COMPANY, LP from 2004 to 2006.



Not a concern.

### 8.2.31 DCP 

In [76]:
print(next(issues))


DCP MIDSTREAM (OPERATOR_ID 31130) has changed its name:

	Was named DCP MIDSTREAM from 2006 to 2018.

	Was named DUKE ENERGY FIELD SERVICES from 2004 to 2005.

	Was named DUKE ENERGY FIELD SERVICES LLC from 2004 to 2004.

	Was named DUKE ENERGY FIELD SERVICES LP from 2004 to 2005.

	Was named DUKE ENERGY FIELD SERVICES, LP from 2004 to 2005.

	Was named DUKE ENERGY NGL OPERATING from 2004 to 2004.

	Was named DUKE ENERGY NGL OPERATING, LLC from 2004 to 2005.

	Was named DUKE ENERGY NGL SERVICES from 2004 to 2004.

	Was named DUKE ENERGY NGL SERVICES, LP from 2004 to 2005.



Since it is the only organization in the group, this is not a concern.

### 3.2.32 BP

In [77]:
print(next(issues))


BP PIPELINE (NORTH AMERICA) INC. (OPERATOR_ID 31189) has changed its name:

	Was named BP PIPELINE from 2006 to 2006.

	Was named BP PIPELINE (NORTH AMERICA) INC. from 2007 to 2018.

	Was named BP PIPELINES N.A. from 2005 to 2005.

	Was named BP PIPELINES NORTH AMERICA INC. from 2004 to 2004.

BP PIPELINE (NORTH AMERICA) INC. (OPERATOR_ID 31189) has the same parent company as:

	BP OIL PIPELINE CO (OPERATOR_ID 18386)

	OLYMPIC PIPE LINE COMPANY (OPERATOR_ID 30781)

	BP WEST COAST PRODUCTS L.L.C. (OPERATOR_ID 31610)

	BLACK LAKE PIPE LINE CO (OPERATOR_ID 1466)

	SEAWAY PRODUCTS PIPELINE COMPANY (OPERATOR_ID 31549)

BP PIPELINE (NORTH AMERICA) INC. (OPERATOR_ID 31189) may have a namesake or namesakes:

	BP PIPELINES NORTH AMERICA INC. (OPERATOR_ID 395)

	BP PIPELINES N.A. (OPERATOR_ID 395)

	BP PIPELINES (NORTH AMERICA),  INC. (OPERATOR_ID 395)

	BP PIPELINES (NORTH AMERICA), INC. (OPERATOR_ID 395)

	BP PIPELINES NORTH AMERICA INC. (OPERATOR_ID 18386)

	BP PIPELINES N.A. (OPERATOR_ID 18

In [78]:
report_names(['31189', '18386', '30781', '31610', '1466', '31549'])


Organization 31189 has changed its name:
    31189 was named Bp Pipeline from 2006 to 2006.
    31189 was named Bp Pipeline North America Inc from 2007 to 2018.
    31189 was named Bp Pipelines NA from 2005 to 2005.
    31189 was named Bp Pipelines North America Inc from 2004 to 2004.

Organization 18386 has changed its name:
    18386 was named Bp Oil from 2006 to 2006.
    18386 was named Bp Oil Pipeline from 2005 to 2005.
    18386 was named Bp Oil Pipeline Co from 2007 to 2018.
    18386 was named Bp Oil Pipeline Company from 2004 to 2004.

Organization 31610 has changed its name:
    31610 was named Bp West Coast Products LLC from 2007 to 2012.
    31610 was named Bp Westcoast Products from 2005 to 2005.
    31610 was named Bp Westcoast Products LLC from 2004 to 2004.
    31610 was named Bp Westcoast Products Llc from 2006 to 2006.

Organization 1466 has changed its name:
    1466 was named Black Lake Pipe Line Co from 2007 to 2009.
    1466 was named Black Lake Pipeline from 200

In [79]:
company_groups = add_company_group('BP (Group)', ['31189', '18386', '31610', '1466', '31549'], company_groups)
company_groups.tail()

,members,name
68,31189,BP (Group)
69,18386,BP (Group)
70,31610,BP (Group)
71,1466,BP (Group)
72,31549,BP (Group)


In [80]:
m_as = add_m_a('BP (Group)', ['BP (Group)', '30781'], m_as, start_year='2001', end_year='2006')
m_as.tail()

,members,name,start_year,end_year
22,31556,Chevron (Group),2002,NaN
23,Chevron (Group),Chevron (Group),2002,2014
24,31554,Chevron (Group),2002,2014
25,BP (Group),BP (Group),2001,2006
26,30781,BP (Group),2001,2006


### 8.2.33 ExxonMobil (repeat)

In [81]:
print(next(issues))


EXXONMOBIL PIPELINE CO (OPERATOR_ID 12628) has changed its name:

	Was named EXXONMOBIL PIPELINE CO from 2007 to 2018.

	Was named MOBIL  PIPE  LINE COMPANY from 2012 to 2018.

	Was named MOBIL PIPE LINE COMPANY from 2004 to 2006.

	Was named MOBIL PIPELINE CO from 2010 to 2010.

EXXONMOBIL PIPELINE CO (OPERATOR_ID 12628) may have a namesake or namesakes:

	EXXONMOBIL PIPELINE COMPANY (OPERATOR_ID 4906)

	MOBIL CORP (OPERATOR_ID 12624)

	MOBIL OIL CORPORATION - MOBIL BEAUMNT REFINERY (HULL) (OPERATOR_ID 12624)

	MOBIL OIL  CORPORATION-MOBIL BEAUMONT REFINERY (MAGPETCO) (OPERATOR_ID 12624)

	MOBIL OIL CORPORATION - MOBIL BEAUMONT REFINERY (HULL) (OPERATOR_ID 12624)

	EXXONMOBIL OIL CORPORATION - BEAUMONT REFINERY (HULL) (OPERATOR_ID 12624)

	EXXONMOBIL OIL CORPORATION - BEAUMONT REFINERY (MAGPETCO) (OPERATOR_ID 12624)

	MOBIL BEAUMONT REFINERY (OPERATOR_ID 12624)

	MOBIL CHEMICAL CO (OPERATOR_ID 12634)

	MOBIL CHEMICAL COMPANY (OPERATOR_ID 12634)

	EXXONMOBIL CHEMICAL COMPANY (OPERATOR

Already covered above.

### 8.2.34 Amoco

In [82]:
print(next(issues))


AMOCO OIL CO (OPERATOR_ID 395) has changed its name:

	Was named AMOCO OIL from 2006 to 2006.

	Was named AMOCO OIL CO from 2007 to 2018.

	Was named AMOCO OIL COMPANY from 2004 to 2005.

AMOCO OIL CO (OPERATOR_ID 395) has the same parent company as:

	DOME PETROLEUM CORP (OPERATOR_ID 3466)



According to our notes, Dome has been part of Amoco since 1988.

In [83]:
company_groups = add_company_group('Amoco (Group)', ['395', '3466'], company_groups)
company_groups.tail()

,members,name
70,31610,BP (Group)
71,1466,BP (Group)
72,31549,BP (Group)
73,395,Amoco (Group)
74,3466,Amoco (Group)


In [84]:
print(next(issues))


DOME PETROLEUM CORP (OPERATOR_ID 3466) has changed its name:

	Was named DOME PETROLEUM CORP from 2007 to 2011.

	Was named DOME PETROLEUM LLC from 2010 to 2010.

	Was named DOME PIPELINE CORP from 2004 to 2004.

	Was named DOME PIPELINE CORPORATION from 2004 to 2006.



Name change is not a concern.

###  8.2.35 Dixie (repeat)

In [85]:
print(next(issues))


DIXIE PIPELINE COMPANY LLC (OPERATOR_ID 3445) has changed its name:

	Was named DIXIE PIPELINE from 2007 to 2010.

	Was named DIXIE PIPELINE COMPANY from 2004 to 2006.

	Was named DIXIE PIPELINE COMPANY LLC from 2011 to 2018.



Name change is not a concern.

### 8.2.36 West shore

In [86]:
print(next(issues))


WEST SHORE PIPELINE CO (OPERATOR_ID 22430) has changed its name:

	Was named WEST SHORE PIPELINE from 2006 to 2006.

	Was named WEST SHORE PIPELINE CO from 2007 to 2018.

	Was named WEST SHORE PIPELINE COMPANY from 2004 to 2006.



Name change is not a concern.

### 8.2.37 Alon USA

In [87]:
print(next(issues))


ALON USA, LP (OPERATOR_ID 31443) may have a namesake or namesakes:

	ALON USA (OPERATOR_ID 26136)



Seems the organization has adapted a new operator ID for some reason.

In [88]:
report_names(['31443', '26136'])

No organizations were renamed at any time.


In [89]:
find_total_miles('31443', range(2010, 2013))

[[2010, 21.0], [2011, 21.0], [2012, 21.0]]

In [90]:
find_total_miles_2004('31443', range(2005, 2009))

[[2005, 320.0], [2006, 21.0], [2007, 21.0], [2008, 21.0]]

In [91]:
find_total_miles('26136', range(2010, 2015))

[[2010, 129.97],
 [2011, 129.97],
 [2012, 129.97],
 [2013, 129.97],
 [2014, 129.97]]

Seems to be two separate subsidiaries.

In [92]:
company_groups = add_company_group('Alon (Group)', ['31443', '26136'], company_groups)

### 8.2.38 Citgo

In [93]:
print(next(issues))


CITGO PRODUCTS PIPELINE CO (OPERATOR_ID 30755) has changed its name:

	Was named CITGO PRODUCTS PIPELINE from 2006 to 2006.

	Was named CITGO PRODUCTS PIPELINE CO from 2005 to 2018.

	Was named CITGO PRODUCTS PIPELINE COMPANY from 2004 to 2004.

CITGO PRODUCTS PIPELINE CO (OPERATOR_ID 30755) has the same parent company as:

	CITGO PIPELINE CO (OPERATOR_ID 2387)

	CITGO REFINING & CHEMICAL CO. L.P. (OPERATOR_ID 31023)

CITGO PRODUCTS PIPELINE CO (OPERATOR_ID 30755) may have a namesake or namesakes:

	CITGO PETROLEUM  (OPERATOR_ID 2387)

	CITGO PETROLEUM (OPERATOR_ID 2387)

	CITGO PETROLEUM CORPORATION (OPERATOR_ID 31023)

	CITGO PETROLEUM COMPANY (OPERATOR_ID 31023)



In [94]:
report_names(['30755', '2387', '31023'])


Organization 30755 has changed its name:
    30755 was named Citgo Products Pipeline from 2006 to 2006.
    30755 was named Citgo Products Pipeline Co from 2005 to 2018.
    30755 was named Citgo Products Pipeline Company from 2004 to 2004.

Organization 2387 has changed its name:
    2387 was named Citgo Pipeline from 2006 to 2006.
    2387 was named Citgo Pipeline Co from 2007 to 2018.
    2387 was named Citgo Pipeline Company from 2004 to 2005.


Our notes suggest no M&A activities.

In [95]:
company_groups = add_company_group('Citgo (Group)', ['30755', '2387', '31023'], company_groups)
company_groups.tail()

,members,name
75,31443,Alon (Group)
76,26136,Alon (Group)
77,30755,Citgo (Group)
78,2387,Citgo (Group)
79,31023,Citgo (Group)


### 8.2.39 Sinclair

In [96]:
print(next(issues))


SINCLAIR TRANSPORTATION COMPANY (OPERATOR_ID 15156) has changed its name:

	Was named SINCLAIR PIPELINE COMPANY from 2004 to 2005.

	Was named SINCLAIR TRANSPORTATION from 2006 to 2006.

	Was named SINCLAIR TRANSPORTATION COMPANY from 2006 to 2018.



Not a concern.

### 8.2.40 Holly

In [97]:
print(next(issues))


HOLLY ENERGY PARTNERS - OPERATING, L.P. (OPERATOR_ID 32011) has changed its name:

	Was named HOLLY ENERGY PARTNERS from 2006 to 2006.

	Was named HOLLY ENERGY PARTNERS - OPERATING, L.P. from 2004 to 2018.

HOLLY ENERGY PARTNERS - OPERATING, L.P. (OPERATOR_ID 32011) has the same parent company as:

	UNEV PIPELINE, LLC (OPERATOR_ID 32493)

	FRONTIER PIPELINE CO (OPERATOR_ID 5656)

	NAVAJO PIPELINE CO (OPERATOR_ID 13161)



In [98]:
report_names(['32011', '32493', '5656', '13161'])


Organization 32011 has changed its name:
    32011 was named Holly Energy Partners from 2006 to 2006.
    32011 was named Holly Energy Partners  Operating LP from 2004 to 2018.

Organization 13161 has changed its name:
    13161 was named Navajo Pipeline Co from 2005 to 2007.
    13161 was named Navajo Pipeline Co Lp from 2004 to 2004.
    13161 was named Navajo Pipeline Company from 2006 to 2006.


In [99]:
company_groups = add_company_group('HollyFrontier (Group)', ['32011', '32493', '5656', '13161'], company_groups)
company_groups.tail()

,members,name
79,31023,Citgo (Group)
80,32011,HollyFrontier (Group)
81,32493,HollyFrontier (Group)
82,5656,HollyFrontier (Group)
83,13161,HollyFrontier (Group)


Repeats

In [100]:
print(next(issues))


ENBRIDGE PIPELINES (SOUTHERN LIGHTS) L.L.C. (OPERATOR_ID 32502) may have a namesake or namesakes:

	ENBRIDGE ENERGY PARTNERS, LP (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, LP  (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, L..P. (OPERATOR_ID 11169)

	ENBRIDGE PIPELINES (NORTH DAKOTA) LLC (OPERATOR_ID 15774)

	ENBRIDGE ENERGY PARTNERS, LP (OPERATOR_ID 15774)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 15774)

	ENBRIDGE (US) INC (OPERATOR_ID 31448)

	ENBRIDGE (U.S.) INC. (OPERATOR_ID 31448)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 31448)

	ENBRIDGE ENERGY PARTNERS, LP (OPERATOR_ID 31947)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 31947)

	ENBRIDGE ENERGY COMPANY INC. (OPERATOR_ID 32080)

	ENBRIDGE ENERGY COMPANY, INC. (OPERATOR_ID 32080)



In [101]:
print(next(issues))


TESORO LOGISTICS NORTHWEST PIPELINE LLC (OPERATOR_ID 39029) may have a namesake or namesakes:

	TESORO - HIGH PLAINS PIPELINE COMPANY (OPERATOR_ID 31570)

	TESORO HIGH PLAINS PIPELINE COMPANY  (OPERATOR_ID 31570)

	TESORO CORPORATION (OPERATOR_ID 31570)

	TESORO REFINING & MARKETING CO., MOUNTAIN REGION (OPERATOR_ID 31583)

	TESORO REFINING & MARKETING CO, MOUNTAIN REGION  (OPERATOR_ID 31583)

	TESORO CORPORATION (OPERATOR_ID 31583)

	TESORO REFINING & MARKETING C O., GOLDEN EAGLE (OPERATOR_ID 31874)

	TESORO CORPORATION (OPERATOR_ID 31874)

	TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA (OPERATOR_ID 38933)



### 8.2.41 Genesis

In [102]:
print(next(issues))


GENESIS PIPELINE TEXAS, L.P. (OPERATOR_ID 31045) has changed its name:

	Was named  GENESIS PIPELINE USA L.P. from 2005 to 2005.

	Was named GENESIS CO2 PIPELINE, L. P. from 2005 to 2005.

	Was named GENESIS CO2, L.P. from 2006 to 2006.

	Was named GENESIS CRUDE OIL LP from 2007 to 2007.

	Was named GENESIS PIPELINE TEXAS, L.P. from 2006 to 2018.

	Was named GENESIS PIPELINE USA LP from 2004 to 2004.

	Was named GENESIS PIPELINE USA, L.P. from 2008 to 2018.

GENESIS PIPELINE TEXAS, L.P. (OPERATOR_ID 31045) has the same parent company as:

	GENESIS PIPELINE TEXAS, L.P. (OPERATOR_ID 32407)



In [103]:
report_names(['31045', '32407'])


Organization 31045 has changed its name:
    31045 was named  Genesis Pipeline Usa LP from 2005 to 2005.
    31045 was named Genesis Co2 LP from 2006 to 2006.
    31045 was named Genesis Co2 Pipeline L P from 2005 to 2005.
    31045 was named Genesis Crude Oil Lp from 2007 to 2007.
    31045 was named Genesis Pipeline Texas LP from 2006 to 2018.
    31045 was named Genesis Pipeline Usa LP from 2008 to 2018.
    31045 was named Genesis Pipeline Usa Lp from 2004 to 2004.


In [104]:
company_groups = add_company_group('Gensis (Group)', ['31045', '32407'], company_groups)
company_groups.tail()

,members,name
81,32493,HollyFrontier (Group)
82,5656,HollyFrontier (Group)
83,13161,HollyFrontier (Group)
84,31045,Gensis (Group)
85,32407,Gensis (Group)


In [105]:
print(next(issues))


EXXONMOBIL OIL CORP - WEST COAST (OPERATOR_ID 26134) has changed its name:

	Was named EXXONMOBIL OIL CORP - WEST COAST from 2007 to 2018.

	Was named EXXONMOBIL OIL CORPORATION - WEST COAST PIPELINE from 2006 to 2006.

	Was named MOBIL OIL CORPORATION - WEST COAST PIPELINE from 2004 to 2004.

	Was named MOBIL OIL CORPORATION WEST COAST PIPELINE from 2005 to 2005.

EXXONMOBIL OIL CORP - WEST COAST (OPERATOR_ID 26134) may have a namesake or namesakes:

	EXXONMOBIL PIPELINE COMPANY (OPERATOR_ID 4906)

	MOBIL CORP (OPERATOR_ID 12624)

	MOBIL OIL CORPORATION - MOBIL BEAUMNT REFINERY (HULL) (OPERATOR_ID 12624)

	MOBIL OIL  CORPORATION-MOBIL BEAUMONT REFINERY (MAGPETCO) (OPERATOR_ID 12624)

	MOBIL OIL CORPORATION - MOBIL BEAUMONT REFINERY (HULL) (OPERATOR_ID 12624)

	EXXONMOBIL OIL CORPORATION - BEAUMONT REFINERY (HULL) (OPERATOR_ID 12624)

	EXXONMOBIL OIL CORPORATION - BEAUMONT REFINERY (MAGPETCO) (OPERATOR_ID 12624)

	MOBIL BEAUMONT REFINERY (OPERATOR_ID 12624)

	MOBIL  PIPE  LINE COMPANY

### 3.2.42 Williams Field services

In [106]:
print(next(issues))


WILLIAMS FIELD SERVICES (OPERATOR_ID 30826) has the same parent company as:

	WILLIAMS FIELD SERVICES - GULF COAST COMPANY, LP (OPERATOR_ID 994)

	WILLIAMS OLEFINS FEEDSTOCK PIPELINES, LLC (OPERATOR_ID 32614)



In [107]:
report_names(['30826', '994', '32614'])


Organization 994 has changed its name:
    994 was named Williams Field Services  Gulf Coast Compant LP from 2005 to 2005.
    994 was named Williams Field Services  Gulf Coast Company LP from 2004 to 2006.
    994 was named Williams Field Services  Gulf Coast Company Lp from 2007 to 2018.


In [108]:
company_groups = add_company_group('Williams Field Services (Group)', ['30826', '994', '32614'], company_groups)
company_groups.tail()

,members,name
84,31045,Gensis (Group)
85,32407,Gensis (Group)
86,30826,Williams Field Services (Group)
87,994,Williams Field Services (Group)
88,32614,Williams Field Services (Group)


In [109]:
print(next(issues))


EXPRESS HOLDINGS (USA), LLC (OPERATOR_ID 31720) has changed its name:

	Was named EXPRESS HOLDINGS (USA), LLC from 2013 to 2018.

	Was named KINDER MORGAN PIPELINES (USA) INC from 2008 to 2012.

	Was named TERASEN PIPELINE (USA) INC  from 2004 to 2004.

	Was named TERASEN PIPELINES (USA) INC. from 2005 to 2007.

EXPRESS HOLDINGS (USA), LLC (OPERATOR_ID 31720) may have a namesake or namesakes:

	KINDER MORGAN ENERGY PARTNERS, L.P. (OPERATOR_ID 4472)

	KINDER MORGAN CANADA INC. (OPERATOR_ID 19585)

	KINDER MORGAN CANAADA INC. (OPERATOR_ID 19585)

	KINDER MORGAN CANADA INC.  (OPERATOR_ID 19585)

	KINDER MORGAN LIQUID TERMINALS, LLC (OPERATOR_ID 26041)

	KINDER MORGAN, INC (OPERATOR_ID 26041)

	KINDER MORGAN LIQUID TERMINAL, LLC (WEST COAST TERMINALS) (OPERATOR_ID 26041)

	KINDER MORGAN LIQUIDS TERMINALS LLC (OPERATOR_ID 26041)

	KINDER MORGAN LIQUIDS TERMINALS, L.P. (OPERATOR_ID 26041)

	KINDER MORGAN WINK PIPELINE LLC (OPERATOR_ID 31957)

	KINDER MORGAN WINK PIPELINE LP (OPERATOR_ID 319

### 8.2.43 Pacific pipeline system

In [110]:
print(next(issues))


PACIFIC PIPELINE SYSTEM LLC (OPERATOR_ID 31325) has changed its name:

	Was named PACIFIC PIPELINE SYSTEM  LLC from 2006 to 2006.

	Was named PACIFIC PIPELINE SYSTEM LLC from 2005 to 2010.

	Was named PACIFIC PIPELINE SYSTEM LLC  from 2004 to 2004.

PACIFIC PIPELINE SYSTEM LLC (OPERATOR_ID 31325) has the same parent company as:

	PACIFIC MARKETING AND TRANSPORTATION LLC (OPERATOR_ID 31695)

	PACIFIC PIPELINE SYSTEM LLC (OPERATOR_ID 31885)

PACIFIC PIPELINE SYSTEM LLC (OPERATOR_ID 31325) may have a namesake or namesakes:

	MOBIL PACIFIC PIPELINE CO (OPERATOR_ID 30005)

	MOBIL PACIFIC PIPELINE COMPANY (OPERATOR_ID 30005)

	MOBIL PACIFIC PIPELINE (OPERATOR_ID 30005)



In [111]:
report_names(['31325', '31695', '31885'])


Organization 31325 has changed its name:
    31325 was named Pacific Pipeline System  Llc from 2006 to 2006.
    31325 was named Pacific Pipeline System Llc from 2005 to 2010.
    31325 was named Pacific Pipeline System Llc  from 2004 to 2004.

Organization 31695 has changed its name:
    31695 was named Pacific Marketing  Transportation  Llc from 2006 to 2006.
    31695 was named Pacific Marketing And Transportation Llc from 2005 to 2007.
    31695 was named Pacific Marketing And Transportation Llc  from 2004 to 2004.

Organization 31885 has changed its name:
    31885 was named Pacific Pipeline System Llc from 2005 to 2010.
    31885 was named Pacific Terminals  Llc from 2006 to 2006.
    31885 was named Pacific Terminals Llc from 2007 to 2008.
    31885 was named Pacific Terminals Llc  from 2004 to 2004.


In [112]:
company_groups = add_company_group('Pacific (Group)', ['31325', '31695', '31885'], company_groups)
company_groups.tail()

,members,name
87,994,Williams Field Services (Group)
88,32614,Williams Field Services (Group)
89,31325,Pacific (Group)
90,31695,Pacific (Group)
91,31885,Pacific (Group)


### 8.2.44 Velero

In [113]:
print(next(issues))


VALERO TERMINALING AND DISTRIBUTION COMPANY (OPERATOR_ID 4430) has the same parent company as:

	VALERO PARTNERS OPERATING CO. LLC (OPERATOR_ID 39105)

	PARKWAY PIPELINE LLC (OPERATOR_ID 32679)

	PREMCOR P A PIPELINE COMPANY (OPERATOR_ID 31415)

	THE PREMCOR PIPELINE COMPANY (OPERATOR_ID 31742)

	VALERO REFINING - TEXAS LP (OPERATOR_ID 32032)

VALERO TERMINALING AND DISTRIBUTION COMPANY (OPERATOR_ID 4430) may have a namesake or namesakes:

	VALERO LOGISTICS OPERATIONS, LP (OPERATOR_ID 31454)

	VALERO LOGISTICS OPERATIONS, LP  (OPERATOR_ID 31454)

	VALERO ENERGY CORPORATION (OPERATOR_ID 31742)



In [114]:
report_names(['4430', '39105', '32679', '31415', '31742', '32032', '31454'])


Organization 31742 has changed its name:
    31742 was named The Premcor Pipeline Co from 2004 to 2005.
    31742 was named The Premcor Pipeline Company from 2006 to 2006.

Organization 31454 has changed its name:
    31454 was named Nustar Logisitics LP from 2006 to 2006.
    31454 was named Nustar Logistics LP from 2007 to 2018.
    31454 was named Valero Logistics Operations Lp from 2004 to 2005.
    31454 was named Valero Logistics Operations Lp  from 2004 to 2004.


Since 31454 was held in a partnership, we do not include this observation in the group. Premcor was acquired by Valero in 2005.

In [115]:
company_groups = add_company_group('Valero (Group)', ['4430', '39105', '32679', '31415', '32032'], company_groups)
company_groups.tail()

,members,name
92,4430,Valero (Group)
93,39105,Valero (Group)
94,32679,Valero (Group)
95,31415,Valero (Group)
96,32032,Valero (Group)


In [116]:
m_as = add_m_a('Valero (Group)', ['Valero (Group)', '31742'], m_as, start_year='2005')
m_as.tail()

,members,name,start_year,end_year
24,31554,Chevron (Group),2002,2014
25,BP (Group),BP (Group),2001,2006
26,30781,BP (Group),2001,2006
27,Valero (Group),Valero (Group),2005,NaN
28,31742,Valero (Group),2005,NaN


In [117]:
print(next(issues))


PHILLIPS 66 COMPANY - SWEENY REFINERY (OPERATOR_ID 15485) has changed its name:

	Was named CONOCOPHILLIPS COMPANY from 2004 to 2010.

	Was named PHILLIPS 66 COMPANY from 2011 to 2011.

	Was named PHILLIPS 66 COMPANY - SWEENY REFINERY from 2012 to 2018.

PHILLIPS 66 COMPANY - SWEENY REFINERY (OPERATOR_ID 15485) may have a namesake or namesakes:

	CONOCOPHILLIPS (OPERATOR_ID 31684)

	CONOCOPHILLIPS - LOS ANGELES REFINERY (OPERATOR_ID 31684)

	CONOCOPHILLIPS PIPE LINE CO. (OPERATOR_ID 31684)

	CONOCOPHILLIPS PIPE LINE COMPANY (OPERATOR_ID 31684)



In [118]:
print(next(issues))


EXXONMOBIL OIL CORPORATION (OPERATOR_ID 12634) has changed its name:

	Was named EXXONMOBIL CHEMICAL COMPANY from 2006 to 2006.

	Was named EXXONMOBIL OIL CORPORATION from 2018 to 2018.

	Was named MOBIL CHEMICAL CO from 2007 to 2017.

	Was named MOBIL CHEMICAL COMPANY from 2004 to 2005.

EXXONMOBIL OIL CORPORATION (OPERATOR_ID 12634) may have a namesake or namesakes:

	EXXONMOBIL PIPELINE COMPANY (OPERATOR_ID 4906)

	MOBIL CORP (OPERATOR_ID 12624)

	MOBIL OIL CORPORATION - MOBIL BEAUMNT REFINERY (HULL) (OPERATOR_ID 12624)

	MOBIL OIL  CORPORATION-MOBIL BEAUMONT REFINERY (MAGPETCO) (OPERATOR_ID 12624)

	MOBIL OIL CORPORATION - MOBIL BEAUMONT REFINERY (HULL) (OPERATOR_ID 12624)

	EXXONMOBIL OIL CORPORATION - BEAUMONT REFINERY (HULL) (OPERATOR_ID 12624)

	EXXONMOBIL OIL CORPORATION - BEAUMONT REFINERY (MAGPETCO) (OPERATOR_ID 12624)

	MOBIL BEAUMONT REFINERY (OPERATOR_ID 12624)

	MOBIL  PIPE  LINE COMPANY (OPERATOR_ID 12628)

	MOBIL PIPELINE CO (OPERATOR_ID 12628)

	MOBIL PIPE LINE COMPA

In [119]:
print(next(issues))


WEST TEXAS GULF PIPELINE CO (OPERATOR_ID 22442) has changed its name:

	Was named WEST TEXAS GULF PIPELINE CO from 2005 to 2018.

	Was named WEST TEXAS GULF PIPELINE COMPANY from 2004 to 2004.



In [120]:
print(next(issues))


BP OIL PIPELINE CO (OPERATOR_ID 18386) has changed its name:

	Was named BP OIL from 2006 to 2006.

	Was named BP OIL PIPELINE from 2005 to 2005.

	Was named BP OIL PIPELINE CO from 2007 to 2018.

	Was named BP OIL PIPELINE COMPANY from 2004 to 2004.

BP OIL PIPELINE CO (OPERATOR_ID 18386) may have a namesake or namesakes:

	BP PIPELINES NORTH AMERICA INC. (OPERATOR_ID 395)

	BP PIPELINES N.A. (OPERATOR_ID 395)

	BP PIPELINES (NORTH AMERICA),  INC. (OPERATOR_ID 395)

	BP PIPELINES (NORTH AMERICA), INC. (OPERATOR_ID 395)

	BP PIPELINES NORTH AMERICA INC. (OPERATOR_ID 30781)

	BP PIPELINES N.A. (OPERATOR_ID 30781)

	BP PIPELINES (NORTH AMERICA), INC. (OPERATOR_ID 30781)

	BP PIPELINES NORTH AMERICA INC. (OPERATOR_ID 31189)

	BP PIPELINES N.A. (OPERATOR_ID 31189)

	BP PIPELINES (NORTH AMERICA), INC. (OPERATOR_ID 31189)

	BP (OPERATOR_ID 3466)

	BP CANADA ENERGY COMPANY (OPERATOR_ID 3466)

	BP PIPELINES NORTH AMERICA INC. (OPERATOR_ID 31270)

	BP PIPELINE (NORTH AMERICA), INC. (OPERATOR_I

In [121]:
print(next(issues))


ALYESKA PIPELINE SERVICE CO (OPERATOR_ID 26149) has changed its name:

	Was named ALYESKA PIPELINE SERVICE CO from 2007 to 2018.

	Was named ALYESKA PIPELINE SERVICE COMPANY from 2004 to 2006.



### 8.2.45 CHS

In [122]:
print(next(issues))


CENEX PIPELINE LLC (OPERATOR_ID 2170) has the same parent company as:

	JAYHAWK PIPELINE LLC (OPERATOR_ID 9175)

	OSAGE PIPE LINE COMPANY, LLC (OPERATOR_ID 14391)

	CHS MCPHERSON REFINERY INC. (OPERATOR_ID 26065)

	SEADRIFT PIPELINE CORP (OPERATOR_ID 26086)

	FRONT RANGE PIPELINE, LLC. (OPERATOR_ID 32283)



In [123]:
report_names(['2170', '9175', '14391', '26065', '26086', '32283'])


Organization 9175 has changed its name:
    9175 was named Jayhawk Pipeline LLC from 2005 to 2006.
    9175 was named Jayhawk Pipeline Llc from 2007 to 2018.
    9175 was named Jayhawk Pipeline Llc  from 2004 to 2004.

Organization 14391 has changed its name:
    14391 was named Osage Pipe Line Company Llc from 2007 to 2018.
    14391 was named Osage Pipeline Company from 2004 to 2004.
    14391 was named Osage Pipeline Company Llc from 2005 to 2010.

Organization 26065 has changed its name:
    26065 was named Chs Mcpherson Refinery Inc from 2015 to 2018.
    26065 was named National Coop Refinery Assco from 2007 to 2014.
    26065 was named National Cooperative  Refinery Association  from 2004 to 2004.
    26065 was named National Cooperative Refinery Association from 2004 to 2004.
    26065 was named National Cooperative Refining Association Ncra from 2005 to 2006.

Organization 26086 has changed its name:
    26086 was named Seadrift Pipeline Corp from 2007 to 2018.
    26086 was 

In [124]:
company_groups = add_company_group('CHS (Group)', ['2170', '9175', '14391', '26065', '26086', '32283'], company_groups)
company_groups.tail()

,members,name
98,9175,CHS (Group)
99,14391,CHS (Group)
100,26065,CHS (Group)
101,26086,CHS (Group)
102,32283,CHS (Group)


In [125]:
print(next(issues))


MAGELLAN AMMONIA PIPELINE, L.P. (OPERATOR_ID 12105) may have a namesake or namesakes:

	Magellan Pipeline Company, L.P. (OPERATOR_ID 22610)

	MAGELLAN PIPELINE COMPANY, LLC (OPERATOR_ID 22610)

	MAGELLAN PIPELINE COMPANY, L.P. (OPERATOR_ID 22610)

	Magellan Pipelines Holdings, L.P. (OPERATOR_ID 31579)

	MAGELLAN PIPELINES HOLDINGS, L.P. (OPERATOR_ID 31579)



### 8.2.46 Rose Rock

In [126]:
print(next(issues))


ROSE ROCK MIDSTREAM L.P. (OPERATOR_ID 31476) has changed its name:

	Was named ROSE ROCK MIDSTREAM L.P. from 2012 to 2018.

	Was named Rose Rock Midstream L.P. from 2011 to 2011.

	Was named SEMCRUDE L.P. from 2006 to 2007.

	Was named SEMCRUDE LP from 2004 to 2005.

	Was named SEMGROUP LP from 2008 to 2010.

	Was named SEMPIPE L.P. from 2006 to 2007.

	Was named SEMPIPE LP from 2004 to 2005.

ROSE ROCK MIDSTREAM L.P. (OPERATOR_ID 31476) has the same parent company as:

	WHITE CLIFFS PIPELINE, LLC (OPERATOR_ID 32288)



In [127]:
company_groups = add_company_group('Rose Rock (Group)', ['31476', '32288'], company_groups)
company_groups.tail()

,members,name
100,26065,CHS (Group)
101,26086,CHS (Group)
102,32283,CHS (Group)
103,31476,Rose Rock (Group)
104,32288,Rose Rock (Group)


### 8.2.47 BKEP

In [128]:
print(next(issues))


BKEP PIPELINE, LLC (OPERATOR_ID 32551) has the same parent company as:

	BKEP CRUDE, LLC (OPERATOR_ID 32481)



In [129]:
report_names(['32551', '32481'])

No organizations were renamed at any time.


In [130]:
company_groups = add_company_group('BKEP (Group)', ['32551', '32481'], company_groups)
company_groups.tail()

,members,name
102,32283,CHS (Group)
103,31476,Rose Rock (Group)
104,32288,Rose Rock (Group)
105,32551,BKEP (Group)
106,32481,BKEP (Group)


In [131]:
print(next(issues))


JAYHAWK PIPELINE LLC (OPERATOR_ID 9175) has changed its name:

	Was named JAYHAWK PIPELINE L.L.C. from 2005 to 2006.

	Was named JAYHAWK PIPELINE LLC from 2007 to 2018.

	Was named JAYHAWK PIPELINE LLC  from 2004 to 2004.



In [132]:
print(next(issues))


SEADRIFT PIPELINE CORP (OPERATOR_ID 26086) has changed its name:

	Was named SEADRIFT PIPELINE CORP from 2007 to 2018.

	Was named SEADRIFT PIPELINE CORPORATION from 2004 to 2006.



In [133]:
print(next(issues))


PLAINS PIPELINE, L.P. (OPERATOR_ID 300) may have a namesake or namesakes:

	TESORO HIGH PLAINS PIPELINE COMPANY LLC (OPERATOR_ID 31570)

	TESORO - HIGH PLAINS PIPELINE COMPANY (OPERATOR_ID 31570)

	TESORO HIGH PLAINS PIPELINE COMPANY  (OPERATOR_ID 31570)

	GREAT SALT PLAINS PIPELINE LLC (OPERATOR_ID 39401)



### 8.2.48 Targa Resources

In [134]:
print(next(issues))


TARGA RESOURCES OPERATING LLC (OPERATOR_ID 32296) has the same parent company as:

	TARGA NGL PIPE LINE CO (OPERATOR_ID 30626)

	TARGA DOWNSTREAM LLC (OPERATOR_ID 39823)

	TARGA MIDSTREAM SERVICES, L.P. (OPERATOR_ID 22175)

	TARGA RESOURCES, INC. (OPERATOR_ID 31977)

TARGA RESOURCES OPERATING LLC (OPERATOR_ID 32296) may have a namesake or namesakes:

	TARGA NGL PIPLINE COMPANY LLC (OPERATOR_ID 30626)



In [135]:
report_names(['32296', '30626', '39823', '22175', '31977'])


Organization 30626 has changed its name:
    30626 was named Dynegy Ngl Pipeline from 2004 to 2004.
    30626 was named Targa Ngl Pipe Line Co from 2007 to 2018.
    30626 was named Targa Ngl Pipeline Company Llc from 2006 to 2006.
    30626 was named Targa Ngl Pipline Company Llc from 2005 to 2005.

Organization 22175 has changed its name:
    22175 was named Dynegy Midstream Services LP  from 2004 to 2004.
    22175 was named Dynegy Midstream Services Limited Partnership from 2004 to 2004.
    22175 was named Dynegy Midstream Services Lp from 2004 to 2004.
    22175 was named Targa Midstream Services LP from 2007 to 2009.
    22175 was named Targa Midstream Services Limited Partnership from 2005 to 2005.
    22175 was named Targa Midstream Services Lp from 2006 to 2006.


Dynegy still exists under different ownership. Targa probably bought the pipeline assets in 2005.

In [136]:
company_groups = add_company_group('Targa (Group)', ['32296', '39823', '31977'], company_groups)
company_groups.tail()

,members,name
105,32551,BKEP (Group)
106,32481,BKEP (Group)
107,32296,Targa (Group)
108,39823,Targa (Group)
109,31977,Targa (Group)


In [137]:
m_as = add_m_a('Dynegy (Group)', ['30626', '22175'], m_as, end_year = '2005')
m_as = add_m_a('Targa (Group)', ['307626', '22175'], m_as, start_year = '2006')
m_as.tail()

,members,name,start_year,end_year
28,31742,Valero (Group),2005,NaN
29,30626,Dynegy (Group),NaN,2005
30,22175,Dynegy (Group),NaN,2005
31,307626,Targa (Group),2006,NaN
32,22175,Targa (Group),2006,NaN


In [138]:
print(next(issues))


ENBRIDGE PIPELINES (OZARK) L.L.C. (OPERATOR_ID 31947) may have a namesake or namesakes:

	ENBRIDGE ENERGY PARTNERS, LP (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, LP  (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, L..P. (OPERATOR_ID 11169)

	ENBRIDGE PIPELINES (NORTH DAKOTA) LLC (OPERATOR_ID 15774)

	ENBRIDGE ENERGY PARTNERS, LP (OPERATOR_ID 15774)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 15774)

	ENBRIDGE (US) INC (OPERATOR_ID 31448)

	ENBRIDGE (U.S.) INC. (OPERATOR_ID 31448)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 31448)

	ENBRIDGE ENERGY COMPANY INC. (OPERATOR_ID 32080)

	ENBRIDGE ENERGY COMPANY, INC. (OPERATOR_ID 32080)



In [139]:
print(next(issues))


HARVEST MIDSTREAM COMPANY (OPERATOR_ID 30782) has changed its name:

	Was named HARVEST MIDSTREAM COMPANY from 2018 to 2018.

	Was named HARVEST PIPELINE COMPANY from 2007 to 2017.



In [140]:
print(next(issues))


COUNTRYMARK REFINING AND LOGISTICS, LLC (OPERATOR_ID 26049) has changed its name:

	Was named CONTRYMARK COOPERATIVE, LLP from 2005 to 2005.

	Was named COUNTRYMARK COOPERATIVE, LLP from 2004 to 2010.

	Was named COUNTRYMARK COOPERATIVE, LLP  from 2004 to 2004.

	Was named COUNTRYMARK REFINING AND LOGISTICS, LLC from 2011 to 2018.



In [141]:
print(next(issues))


CHEVRON MIDSTREAM PIPELINES LLC (OPERATOR_ID 31556) has changed its name:

	Was named CHEVRON MIDSTREAM PIPELINES LLC from 2009 to 2015.

	Was named TEXACO PIPELINES LLC from 2004 to 2006.

	Was named TEXACO PIPELINES, LLC from 2007 to 2008.

CHEVRON MIDSTREAM PIPELINES LLC (OPERATOR_ID 31556) may have a namesake or namesakes:

	CHEVRON PIPELINE COMPANY (OPERATOR_ID 2731)

	CHEVRON PIPE LINE COMPANY (OPERATOR_ID 2731)

	CHEVRON PETROCHEMICAL PIPELINE LLC (OPERATOR_ID 31554)

	CHEVRON PETROCHEMICAL PIPELINE LLC (FORMERLY TEXACO PETROCHEMICAL PIPELINE LLC) (OPERATOR_ID 31554)

	CHEVRON USA, INC. (OPERATOR_ID 2339)

	CHEVRON USA,  INC. (OPERATOR_ID 2339)

	CHEVRON PRODUCTS COMPANY - HAWAII  (OPERATOR_ID 2339)

	TEXACO PETROCHEMICAL PIPELINE LLC (OPERATOR_ID 31554)

	CHEVRON PETROCHEMICAL PIPELINE LLC (FORMERLY TEXACO PETROCHEMICAL PIPELINE LLC) (OPERATOR_ID 31554)



### 3.2.49 Dow

In [142]:
print(next(issues))


DOW PIPELINE CO (OPERATOR_ID 3527) has changed its name:

	Was named DOW PIPELINE CO from 2007 to 2018.

	Was named DOW PIPELINE COMPANY from 2004 to 2006.

DOW PIPELINE CO (OPERATOR_ID 3527) has the same parent company as:

	DOW PIPELINE CO - CAYUSE (OPERATOR_ID 2162)

	DOW PIPELINE CO (OPERATOR_ID 3535)

	THE DOW CHEMICAL COMPANY (OPERATOR_ID 30959)

DOW PIPELINE CO (OPERATOR_ID 3527) may have a namesake or namesakes:

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 2162)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 2162)

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 3535)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 3535)

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 26086)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 26086)

	TARGA DOWNSTREAM LLC (OPERATOR_ID 39823)



In [143]:
report_names(['3527', '2162', '3535', '30959', '26086', '39823'])


Organization 3527 has changed its name:
    3527 was named Dow Pipeline Co from 2007 to 2018.
    3527 was named Dow Pipeline Company from 2004 to 2006.

Organization 2162 has changed its name:
    2162 was named Dow Pipeline Co  Cayuse from 2007 to 2018.
    2162 was named Dow Pipeline Company from 2004 to 2006.

Organization 3535 has changed its name:
    3535 was named Dow Pipeline Co from 2007 to 2018.
    3535 was named Dow Pipeline Company from 2004 to 2006.

Organization 26086 has changed its name:
    26086 was named Seadrift Pipeline Corp from 2007 to 2018.
    26086 was named Seadrift Pipeline Corporation from 2004 to 2006.


In [144]:
company_groups = add_company_group('Dow (Group)', ['3527', '2162', '3535', '30959'], company_groups)
company_groups.tail()

,members,name
109,31977,Targa (Group)
110,3527,Dow (Group)
111,2162,Dow (Group)
112,3535,Dow (Group)
113,30959,Dow (Group)


### 8.2.50 Boardwalk

In [145]:
print(next(issues))


BOARDWALK LOUISIANA MIDSTREAM, LLC (OPERATOR_ID 39138) has the same parent company as:

	BOARDWALK PETROCHEMICAL PIPELINE, LLC (OPERATOR_ID 31554)



In [146]:
report_names(['39138', '31554'])


Organization 31554 has changed its name:
    31554 was named Boardwalk Petrochemical Pipeline Llc from 2014 to 2018.
    31554 was named Chevron Petrochemical Pipeline Llc from 2007 to 2013.
    31554 was named Chevron Petrochemical Pipeline Llc Formerly Texaco Petrochemical Pipeline Llc from 2006 to 2006.
    31554 was named Texaco Petrochemical Pipeline Llc from 2004 to 2005.


In [147]:
company_groups = add_company_group('Boardwalk (Group)', ['39138'], company_groups)
company_groups.tail()

,members,name
110,3527,Dow (Group)
111,2162,Dow (Group)
112,3535,Dow (Group)
113,30959,Dow (Group)
114,39138,Boardwalk (Group)


In [148]:
m_as = add_m_a('Boardwalk (Group)', ['Boardwalk (Group)', '31554'], m_as, start_year = '2014')
m_as.tail()

,members,name,start_year,end_year
30,22175,Dynegy (Group),NaN,2005
31,307626,Targa (Group),2006,NaN
32,22175,Targa (Group),2006,NaN
33,Boardwalk (Group),Boardwalk (Group),2014,NaN
34,31554,Boardwalk (Group),2014,NaN


In [149]:
print(next(issues))


VALERO PARTNERS OPERATING CO. LLC (OPERATOR_ID 39105) may have a namesake or namesakes:

	VALERO ENERGY CORPORATION (OPERATOR_ID 4430)

	VALERO LOGISTICS OPERATIONS, LP (OPERATOR_ID 31454)

	VALERO LOGISTICS OPERATIONS, LP  (OPERATOR_ID 31454)

	VALERO ENERGY CORPORATION (OPERATOR_ID 31742)



In [150]:
print(next(issues))


MAGELLAN CRUDE OIL PIPELINE COMPANY, L.P. (OPERATOR_ID 39504) may have a namesake or namesakes:

	Magellan Ammonia Pipeline, L.P. (OPERATOR_ID 12105)

	Magellan Pipeline Company, L.P. (OPERATOR_ID 22610)

	MAGELLAN PIPELINE COMPANY, LLC (OPERATOR_ID 22610)

	MAGELLAN PIPELINE COMPANY, L.P. (OPERATOR_ID 22610)

	Magellan Pipelines Holdings, L.P. (OPERATOR_ID 31579)

	MAGELLAN PIPELINES HOLDINGS, L.P. (OPERATOR_ID 31579)



In [151]:
print(next(issues))


TARGA NGL PIPE LINE CO (OPERATOR_ID 30626) has changed its name:

	Was named DYNEGY NGL PIPELINE from 2004 to 2004.

	Was named TARGA NGL PIPE LINE CO from 2007 to 2018.

	Was named TARGA NGL PIPELINE COMPANY, LLC from 2006 to 2006.

	Was named TARGA NGL PIPLINE COMPANY LLC from 2005 to 2005.

TARGA NGL PIPE LINE CO (OPERATOR_ID 30626) may have a namesake or namesakes:

	DYNEGY, INC (OPERATOR_ID 22175)

	TARGA RESOURCES OPERATING LP (OPERATOR_ID 32296)

	TARGA RESOURCES INC. (OPERATOR_ID 32296)



In [152]:
print(next(issues))


PETROLOGISTICS OLEFINS, LLC (OPERATOR_ID 20160) has changed its name:

	Was named PETROLOGISTICS OLEFINS LLC from 2005 to 2005.

	Was named PETROLOGISTICS OLEFINS, LLC from 2007 to 2012.

	Was named PETROLOGISTICS OLEFINS, LLC  from 2004 to 2004.

	Was named PETROLOGISTICS OLEFINS, LLC. from 2006 to 2006.

	Was named PL Midstream LLC from 2010 to 2010.



### 8.2.51 Enlink

In [153]:
print(next(issues))


ENLINK LBU-GAS&LIQUIDS (OPERATOR_ID 32005) has the same parent company as:

	ENLINK NGL PIPELINE, LP (OPERATOR_ID 32107)



In [154]:
report_names(['32005', '32107'])


Organization 32107 has changed its name:
    32107 was named Crosstex Ngl Pipeline LP from 2006 to 2012.
    32107 was named Enlink Ngl Pipeline Lp from 2013 to 2016.


No indicator of an M&A taking place on LexisNexis.

In [155]:
company_groups = add_company_group('Enlink (Group)', ['32005', '32107'], company_groups)
company_groups.tail()

,members,name
112,3535,Dow (Group)
113,30959,Dow (Group)
114,39138,Boardwalk (Group)
115,32005,Enlink (Group)
116,32107,Enlink (Group)


### 8.2.52 Hunt

In [156]:
print(next(issues))


HUNT REFINING CO (OPERATOR_ID 26048) has changed its name:

	Was named  HUNT REFINING COMPANY from 2004 to 2004.

	Was named HUNT CRUDE OIL SUPPLY COMPANY from 2004 to 2006.

	Was named HUNT REFINING CO from 2007 to 2018.

	Was named HUNT REFINING COMPANY from 2005 to 2006.

HUNT REFINING CO (OPERATOR_ID 26048) has the same parent company as:

	HUNT CRUDE OIL SUPPLY CO (OPERATOR_ID 7660)

HUNT REFINING CO (OPERATOR_ID 26048) may have a namesake or namesakes:

	HUNT CRUDE OIL SUPPLY COMPANY (OPERATOR_ID 7660)



In [157]:
report_names(['26048', '7660'])


Organization 26048 has changed its name:
    26048 was named  Hunt Refining Company from 2004 to 2004.
    26048 was named Hunt Crude Oil Supply Company from 2004 to 2006.
    26048 was named Hunt Refining Co from 2007 to 2018.
    26048 was named Hunt Refining Company from 2005 to 2006.

Organization 7660 has changed its name:
    7660 was named Hunt Crude Oil Supply Co from 2007 to 2018.
    7660 was named Hunt Crude Oil Supply Company from 2004 to 2006.

Organization 26048 has changed its name:
    26048 was named  Hunt Refining Company from 2004 to 2004.
    26048 was named Hunt Crude Oil Supply Company from 2004 to 2006.
    26048 was named Hunt Refining Co from 2007 to 2018.
    26048 was named Hunt Refining Company from 2005 to 2006.


In [158]:
company_groups = add_company_group('Hunt (Group)', ['26048', '7660'], company_groups)
company_groups.tail()

,members,name
115,32005,Enlink (Group)
116,32107,Enlink (Group)
117,26048,Hunt (Group)
118,7660,Hunt (Group)
119,26048,Hunt (Group)


In [159]:
print(next(issues))


TPC GROUP, LLC (OPERATOR_ID 19319) has changed its name:

	Was named TEXAS PETROCHEMICALS CORP from 2007 to 2011.

	Was named TEXAS PETROCHEMICALS LP from 2004 to 2004.

	Was named TEXAS PETROCHEMICALS, LP from 2006 to 2006.

	Was named TPC GROUP, LLC from 2011 to 2018.

	Was named TPC Group LLC from 2010 to 2010.

TPC GROUP, LLC (OPERATOR_ID 19319) may have a namesake or namesakes:

	WEST TEXAS GULF PIPELINE CO (OPERATOR_ID 22442)

	WEST TEXAS GULF PIPELINE COMPANY (OPERATOR_ID 22442)

	DELEK PIPELINE TEXAS, INC (OPERATOR_ID 26061)

	DELEK PIPELINE TEXAS, INC. (OPERATOR_ID 26061)

	TEXAS EASTMAN DIVISION, EASTMAN CHEMICAL CO (OPERATOR_ID 26103)

	GENESIS PIPELINE TEXAS LP (OPERATOR_ID 31045)

	GENESIS PIPELINE TEXAS, L.P. (OPERATOR_ID 32407)

	ASCEND PERFORMANCE MATERIALS TEXAS INC (OPERATOR_ID 31517)

	TEXAS GENCO LP (OPERATOR_ID 31802)

	VALERO REFINING - TEXAS LP (OPERATOR_ID 32032)



In [160]:
print(next(issues))


THE PREMCOR PIPELINE COMPANY (OPERATOR_ID 31742) has changed its name:

	Was named THE PREMCOR PIPELINE CO from 2004 to 2004.

	Was named THE PREMCOR PIPELINE CO. from 2005 to 2005.

	Was named THE PREMCOR PIPELINE COMPANY from 2006 to 2006.

THE PREMCOR PIPELINE COMPANY (OPERATOR_ID 31742) may have a namesake or namesakes:

	THE PREMCOR REFINING GROUP INC (OPERATOR_ID 31415)



### 8.2.53 Eastman Chemical

In [161]:
print(next(issues))


MUSTANG  PIPE  LINE  LLC (OPERATOR_ID 31166) has changed its name:

	Was named MUSTANG  PIPE  LINE  LLC from 2012 to 2018.

	Was named MUSTANG PIPE LINE LLC from 2006 to 2006.

	Was named MUSTANG PIPE LINE PARTNERS from 2004 to 2011.

	Was named MUSTANG PIPELINE PARTNERS from 2005 to 2005.

MUSTANG  PIPE  LINE  LLC (OPERATOR_ID 31166) has the same parent company as:

	TEXAS EASTMAN DIVISION, EASTMAN CHEMICAL CO (OPERATOR_ID 26103)



In [162]:
company_groups = add_company_group('Eastman Chemical (Group)', ['31166', '26103'], company_groups)
company_groups.tail()

,members,name
117,26048,Hunt (Group)
118,7660,Hunt (Group)
119,26048,Hunt (Group)
120,31166,Eastman Chemical (Group)
121,26103,Eastman Chemical (Group)


In [163]:
print(next(issues))


TARGA MIDSTREAM SERVICES, L.P. (OPERATOR_ID 22175) has changed its name:

	Was named DYNEGY MIDSTREAM SERVICES, L.P.  from 2004 to 2004.

	Was named DYNEGY MIDSTREAM SERVICES, LIMITED PARTNERSHIP from 2004 to 2004.

	Was named DYNEGY MIDSTREAM SERVICES, LP from 2004 to 2004.

	Was named TARGA MIDSTREAM SERVICES LIMITED PARTNERSHIP from 2005 to 2005.

	Was named TARGA MIDSTREAM SERVICES LP from 2006 to 2006.

	Was named TARGA MIDSTREAM SERVICES, L.P. from 2007 to 2009.

	Was named TARGA MIDSTREAM SERVICES, LP from 2006 to 2006.

TARGA MIDSTREAM SERVICES, L.P. (OPERATOR_ID 22175) may have a namesake or namesakes:

	DYNEGY INC (OPERATOR_ID 30626)

	TARGA NGL PIPLINE COMPANY LLC (OPERATOR_ID 30626)

	TARGA RESOURCES OPERATING LP (OPERATOR_ID 32296)

	TARGA RESOURCES INC. (OPERATOR_ID 32296)



In [164]:
print(next(issues))


EXXONMOBIL OIL CORPORATION (OPERATOR_ID 12624) has changed its name:

	Was named EXXONMOBIL OIL CORPORATION from 2018 to 2018.

	Was named EXXONMOBIL OIL CORPORATION - BEAUMONT REFINERY (HULL) from 2006 to 2006.

	Was named EXXONMOBIL OIL CORPORATION - BEAUMONT REFINERY (MAGPETCO) from 2006 to 2006.

	Was named MOBIL CORP from 2004 to 2017.

	Was named MOBIL OIL  CORPORATION-MOBIL BEAUMONT REFINERY (MAGPETCO) from 2004 to 2004.

	Was named MOBIL OIL CORPORATION - MOBIL BEAUMNT REFINERY (HULL) from 2004 to 2004.

	Was named MOBIL OIL CORPORATION - MOBIL BEAUMONT REFINERY (HULL) from 2005 to 2005.

	Was named MOBIL OIL CORPORATION - MOBIL BEAUMONT REFINERY (MAGPETCO) from 2005 to 2005.

EXXONMOBIL OIL CORPORATION (OPERATOR_ID 12624) may have a namesake or namesakes:

	EXXONMOBIL PIPELINE COMPANY (OPERATOR_ID 4906)

	MOBIL  PIPE  LINE COMPANY (OPERATOR_ID 12628)

	MOBIL PIPELINE CO (OPERATOR_ID 12628)

	MOBIL PIPE LINE COMPANY (OPERATOR_ID 12628)

	MOBIL CHEMICAL CO (OPERATOR_ID 12634)



In [165]:
print(next(issues))


NORTH DAKOTA PIPELINE COMPANY LLC (OPERATOR_ID 15774) has changed its name:

	Was named ENBRIDGE PIPELINES (NORTH DAKOTA) LLC from 2004 to 2012.

	Was named NORTH DAKOTA PIPELINE COMPANY LLC from 2013 to 2018.

NORTH DAKOTA PIPELINE COMPANY LLC (OPERATOR_ID 15774) may have a namesake or namesakes:

	ENBRIDGE ENERGY, LIMITED PARTNERSHIP (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, LP (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, LP  (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, L..P. (OPERATOR_ID 11169)

	ENBRIDGE PIPELINES (TOLEDO) INC (OPERATOR_ID 31448)

	ENBRIDGE (US) INC (OPERATOR_ID 31448)

	ENBRIDGE (U.S.) INC. (OPERATOR_ID 31448)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 31448)

	ENBRIDGE PIPELINES (OZARK) L.L.C. (OPERATOR_ID 31947)

	ENBRIDGE ENERGY PARTNERS, LP (OPERATOR_ID 31947)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 31947)

	ENBRIDGE ENERGY COMPANY INC. (OPERATOR_ID 32080)

	ENBRIDGE ENERGY COMPANY, INC

In [166]:
print(next(issues))


MARKWEST LIBERTY MIDSTREAM & RESOURCES, L.L.C. (OPERATOR_ID 32412) may have a namesake or namesakes:

	MARKWEST RANGER PIPELINE COMPANY, L.L.C. (OPERATOR_ID 26026)

	MARKWEST RANGER PIPELINE COMPANY, LLC (OPERATOR_ID 26026)

	MARKWEST ENERGY APPALACHIA, LLC (OPERATOR_ID 26026)

	MARKWEST HYDROCARBON, INC (OPERATOR_ID 26026)

	MARKWEST HYDROCARBON, INC. (OPERATOR_ID 26026)

	MARKWEST ENERGY PARTNERS, L.P. (OPERATOR_ID 26026)

	MARKWEST MICHIGAN PIPELINE, LLC (OPERATOR_ID 31871)

	MARKWEST MICHIGAN PIPELINE COMPANY, L.L.C. (OPERATOR_ID 31871)

	MARKWEST-MICHIGAN PIPELINE COMPANY, L.L.C (OPERATOR_ID 31871)

	MARKWEST MICHIGAN PIPELINE COMPANY L.L.C (OPERATOR_ID 31871)



In [167]:
print(next(issues))


MARTIN OPERATING PARTNERSHIP, L.P. (OPERATOR_ID 32051) has changed its name:

	Was named MARTIN OPERATING PARTNERS, LP from 2006 to 2006.

	Was named MARTIN OPERATING PARTNERSHIP LP from 2005 to 2005.

	Was named MARTIN OPERATING PARTNERSHIP, L.P. from 2007 to 2018.



In [168]:
print(next(issues))


CENTRAL FLORIDA PIPELINE CORP (OPERATOR_ID 2190) has changed its name:

	Was named CENTRAL FLORIDA PIPELINE CORP from 2007 to 2018.

	Was named CENTRAL FLORIDA PIPELINE LLC from 2004 to 2006.



In [169]:
print(next(issues))


P B ENERGY STORAGE SERVICES INC. (OPERATOR_ID 31890) has changed its name:

	Was named P B ENERGY STORAGE SERVICES INC. from 2004 to 2009.

	Was named PB ENERGY STORAGE SERVICES INC from 2004 to 2004.

	Was named PB ENERGY STORAGE SERVICES, INC from 2005 to 2005.

	Was named PB ENERGY STORAGE SERVICES, INC. from 2006 to 2006.



In [170]:
print(next(issues))


TARGA RESOURCES, INC. (OPERATOR_ID 31977) may have a namesake or namesakes:

	TARGA NGL PIPLINE COMPANY LLC (OPERATOR_ID 30626)

	TARGA RESOURCES, INC. (OPERATOR_ID 30626)

	TARGA RESOURCES OPERATING LP (OPERATOR_ID 32296)

	TARGA RESOURCES, INC. (OPERATOR_ID 32296)

	TARGA RESOURCES INC. (OPERATOR_ID 32296)

	TARGA RESOURCES, INC. (OPERATOR_ID 22175)



In [171]:
print(next(issues))


DOMINION ENERGY TRANSMISSION, INC. (OPERATOR_ID 2714) has changed its name:

	Was named DOMINION ENERGY TRANSMISSION, INC. from 2017 to 2018.

	Was named DOMINION TRANSMISION, INCORPORATED from 2004 to 2004.

	Was named DOMINION TRANSMISSION INC from 2007 to 2007.

	Was named DOMINION TRANSMISSION, INC from 2008 to 2016.

	Was named DOMINION TRANSMISSION, INC. from 2005 to 2006.

	Was named DOMINION TRANSMISSION, INCORPORATED from 2004 to 2004.



In [172]:
print(next(issues))


TESORO HIGH PLAINS PIPELINE COMPANY LLC (OPERATOR_ID 31570) has changed its name:

	Was named TESORO - HIGH PLAINS PIPELINE COMPANY from 2007 to 2011.

	Was named TESORO HIGH PLAIN PIPELINE COMPANY from 2006 to 2006.

	Was named TESORO HIGH PLAINS PIPELINE COMPANY from 2005 to 2005.

	Was named TESORO HIGH PLAINS PIPELINE COMPANY  from 2004 to 2004.

	Was named TESORO HIGH PLAINS PIPELINE COMPANY LLC from 2012 to 2017.

TESORO HIGH PLAINS PIPELINE COMPANY LLC (OPERATOR_ID 31570) may have a namesake or namesakes:

	TESORO REFINING & MARKETING CO., MOUNTAIN REGION (OPERATOR_ID 31583)

	TESORO REFINING & MARKETING CO, MOUNTAIN REGION  (OPERATOR_ID 31583)

	TESORO CORPORATION (OPERATOR_ID 31583)

	TESORO REFINING & MARKETING C O., GOLDEN EAGLE (OPERATOR_ID 31874)

	TESORO CORPORATION (OPERATOR_ID 31874)

	TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA (OPERATOR_ID 38933)



In [173]:
print(next(issues))


HUNTSMAN PETROCHEMICAL LLC (OPERATOR_ID 30709) has changed its name:

	Was named HUNTSMAN PETROCHEMICAL CORPORATION from 2004 to 2009.

	Was named HUNTSMAN PETROCHEMICAL LLC from 2010 to 2018.



In [174]:
print(next(issues))


WESTERN REFINING LOGISTICS, LP (OPERATOR_ID 31574) has changed its name:

	Was named GIANT PIPELINE COMPANY from 2005 to 2006.

	Was named WESTERN REFINING LOGISTICS, LP from 2018 to 2018.

	Was named WESTERN REFINING PIPELINE COMPANY from 2007 to 2012.

	Was named WESTERN REFINING PIPELINE, LLC from 2013 to 2017.

WESTERN REFINING LOGISTICS, LP (OPERATOR_ID 31574) may have a namesake or namesakes:

	GIANT INDUSTRIES INC (OPERATOR_ID 2371)

	GIANT INDUSTRIES, INC. (OPERATOR_ID 2371)

	GIANT INDUSTRIES ARIZONA, INC. (OPERATOR_ID 2371)

	WESTERN REFINING SOUTHWEST, INC (OPERATOR_ID 2371)



### 8.2.54 Delek

In [175]:
print(next(issues))


DELEK LOGISTICS OPERATING, LLC. (OPERATOR_ID 11551) has changed its name:

	Was named DELEK LOGISTICS OPERATING, LLC. from 2013 to 2018.

	Was named LION OIL CO from 2005 to 2008.

	Was named LION OIL TRADING & TRANSPORTATION, INC from 2009 to 2012.

	Was named LION OIL TRADING & TRANSPORTATION, INC. from 2004 to 2004.

DELEK LOGISTICS OPERATING, LLC. (OPERATOR_ID 11551) has the same parent company as:

	DELEK MARKETING AND SUPPLY, LP (OPERATOR_ID 15851)

	DELEK CRUDE LOGISTICS, LLC. (OPERATOR_ID 26061)

	PARAMOUNT PETROLEUM CORP (OPERATOR_ID 26136)

DELEK LOGISTICS OPERATING, LLC. (OPERATOR_ID 11551) may have a namesake or namesakes:

	DELEK MARKETING & SUPPLY, LP (OPERATOR_ID 15851)

	DELEK PIPELINE TEXAS, INC (OPERATOR_ID 26061)

	DELEK PIPELINE TEXAS, INC. (OPERATOR_ID 26061)

	DELEK US HOLDINGS (OPERATOR_ID 26061)

	DELEK U.S. HOLDINGS (OPERATOR_ID 26061)

	MEDALLION OPERATING COMPANY, LLC (OPERATOR_ID 39183)



In [176]:
report_names(['11551', '15851', '26061', '26136', '39183'])


Organization 11551 has changed its name:
    11551 was named Delek Logistics Operating Llc from 2013 to 2018.
    11551 was named Lion Oil Co from 2005 to 2008.
    11551 was named Lion Oil Trading  Transportation Inc from 2004 to 2012.

Organization 15851 has changed its name:
    15851 was named Delek Marketing  Supply Lp from 2006 to 2006.
    15851 was named Delek Marketing And Supply Lp from 2007 to 2018.
    15851 was named Pride Refining Inc from 2004 to 2005.

Organization 26061 has changed its name:
    26061 was named Delek Crude Logistics Llc from 2013 to 2018.
    26061 was named Delek Pipeline Texas Inc from 2004 to 2012.
    26061 was named Mcmurrey Pipeline Co from 2008 to 2008.


LexisNexis does not indicate that Delek bought Lion Oil.

In [177]:
company_groups = add_company_group('Delek (Group)', ['11551', '15851', '26061', '26136'], company_groups)
company_groups.tail()

,members,name
121,26103,Eastman Chemical (Group)
122,11551,Delek (Group)
123,15851,Delek (Group)
124,26061,Delek (Group)
125,26136,Delek (Group)


In [178]:
print(next(issues))


THE DOW CHEMICAL COMPANY (OPERATOR_ID 30959) may have a namesake or namesakes:

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 2162)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 2162)

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 3527)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 3527)

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 3535)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 3535)

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 26086)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 26086)

	TARGA DOWNSTREAM LLC (OPERATOR_ID 39823)



### 8.2.55 Suncor

In [179]:
print(next(issues))


PORTLAND PIPE LINE CORPORATION (OPERATOR_ID 15786) has changed its name:

	Was named PORTLAND PIPE LINE CORPORATION from 2004 to 2018.

	Was named PORTLAND PIPELINE CORP from 2007 to 2011.

PORTLAND PIPE LINE CORPORATION (OPERATOR_ID 15786) has the same parent company as:

	SUNCOR ENERGY (USA) PIPELINE CO. (OPERATOR_ID 31822)



In [180]:
company_groups = add_company_group('Suncor (Group)', ['15786', '31822'], company_groups)
company_groups.tail()

,members,name
123,15851,Delek (Group)
124,26061,Delek (Group)
125,26136,Delek (Group)
126,15786,Suncor (Group)
127,31822,Suncor (Group)


### 8.2.56 Arrow

In [181]:
print(next(issues))


ARROW PIPELINE (OPERATOR_ID 39083) has the same parent company as:

	CRESTWOOD MIDSTREAM PARTNERS LP (OPERATOR_ID 39368)



In [182]:
report_names(['39083', '39368'])

No organizations were renamed at any time.


In [183]:
company_groups = add_company_group('Crestwood (Group)', ['39083', '39368'], company_groups)

### 8.2.57 Torrance

In [184]:
print(next(issues))


TORRANCE VALLEY PIPELINE COMPANY LLC (OPERATOR_ID 39534) has the same parent company as:

	COLLINS PIPELINE CO (OPERATOR_ID 26120)

	MOEM PIPELINE LLC (OPERATOR_ID 31167)

	TORRANCE PIPELINE COMPANY LLC (OPERATOR_ID 39535)



In [185]:
report_names(['39534', '26120', '31167', '39535'])


Organization 26120 has changed its name:
    26120 was named Collins Pipeline Co from 2007 to 2018.
    26120 was named Collins Pipeline Company from 2005 to 2006.
    26120 was named Collins Pipeline Llc from 2004 to 2004.


In [186]:
company_groups = add_company_group('Torrance (Group)', ['39534', '26120', '31167', '39535'], company_groups)
company_groups.tail()

,members,name
129,39368,Crestwood (Group)
130,39534,Torrance (Group)
131,26120,Torrance (Group)
132,31167,Torrance (Group)
133,39535,Torrance (Group)


In [187]:
print(next(issues))


CHS MCPHERSON REFINERY INC. (OPERATOR_ID 26065) has changed its name:

	Was named CHS MCPHERSON REFINERY INC. from 2015 to 2018.

	Was named NATIONAL COOP REFINERY ASSCO from 2007 to 2014.

	Was named NATIONAL COOPERATIVE  REFINERY ASSOCIATION  from 2004 to 2004.

	Was named NATIONAL COOPERATIVE REFINERY ASSOCIATION from 2004 to 2004.

	Was named NATIONAL COOPERATIVE REFINING ASSOCIATION (NCRA) from 2005 to 2006.



In [188]:
print(next(issues))


MARKWEST RANGER PIPELINE COMPANY, L.L.C. (OPERATOR_ID 26026) has changed its name:

	Was named MARKWEST ENERGY APPALACHIA, L.L.C. from 2005 to 2006.

	Was named MARKWEST ENERGY APPALACHIA, LLC from 2004 to 2010.

	Was named MARKWEST RANGER PIPELINE COMPANY, L.L.C. from 2012 to 2018.

	Was named MARKWEST RANGER PIPELINE COMPANY, LLC from 2011 to 2011.

MARKWEST RANGER PIPELINE COMPANY, L.L.C. (OPERATOR_ID 26026) may have a namesake or namesakes:

	MARKWEST MICHIGAN PIPELINE, LLC (OPERATOR_ID 31871)

	MARKWEST-MICHIGAN PIPELINE COMPANY, L.L.C (OPERATOR_ID 31871)

	MARKWEST MICHIGAN PIPELINE COMPANY L.L.C (OPERATOR_ID 31871)

	MARKWEST LIBERTY MIDSTREAM & RESOURCES, LLC (OPERATOR_ID 32412)

	MARKWEST LIBERTY MIDSTREAM & RESOURCES, L.L.C. (OPERATOR_ID 32412)



In [189]:
print(next(issues))


BELLE FOURCHE PIPELINE CO (OPERATOR_ID 1248) has changed its name:

	Was named BELLE FOURCHE PIPELINE from 2004 to 2005.

	Was named BELLE FOURCHE PIPELINE CO from 2007 to 2018.

	Was named BELLE FOURCHE PIPELINE CO. from 2006 to 2006.



In [190]:
print(next(issues))


BP WEST COAST PRODUCTS L.L.C. (OPERATOR_ID 31610) has changed its name:

	Was named BP WEST COAST PRODUCTS L.L.C. from 2007 to 2012.

	Was named BP WESTCOAST PRODUCTS from 2005 to 2005.

	Was named BP WESTCOAST PRODUCTS L.L.C. from 2004 to 2004.

	Was named BP WESTCOAST PRODUCTS LLC from 2006 to 2006.

BP WEST COAST PRODUCTS L.L.C. (OPERATOR_ID 31610) may have a namesake or namesakes:

	BP PIPELINES NORTH AMERICA INC. (OPERATOR_ID 395)

	BP PIPELINES N.A. (OPERATOR_ID 395)

	BP PIPELINES (NORTH AMERICA),  INC. (OPERATOR_ID 395)

	BP PIPELINES (NORTH AMERICA), INC. (OPERATOR_ID 395)

	BP PIPELINES NORTH AMERICA INC. (OPERATOR_ID 18386)

	BP PIPELINES N.A. (OPERATOR_ID 18386)

	BP PIPELINES (NORTH AMERICA), INC. (OPERATOR_ID 18386)

	BP PIPELINES NORTH AMERICA INC. (OPERATOR_ID 30781)

	BP PIPELINES N.A. (OPERATOR_ID 30781)

	BP PIPELINES (NORTH AMERICA), INC. (OPERATOR_ID 30781)

	BP PIPELINES NORTH AMERICA INC. (OPERATOR_ID 31189)

	BP PIPELINES N.A. (OPERATOR_ID 31189)

	BP PIPELINES

In [191]:
print(next(issues))


DOW PIPELINE CO - CAYUSE (OPERATOR_ID 2162) has changed its name:

	Was named DOW PIPELINE CO - CAYUSE from 2007 to 2018.

	Was named DOW PIPELINE COMPANY from 2004 to 2006.

DOW PIPELINE CO - CAYUSE (OPERATOR_ID 2162) may have a namesake or namesakes:

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 3527)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 3527)

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 3535)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 3535)

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 26086)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 26086)

	TARGA DOWNSTREAM LLC (OPERATOR_ID 39823)



In [192]:
print(next(issues))


BLACK LAKE PIPE LINE CO (OPERATOR_ID 1466) has changed its name:

	Was named BLACK LAKE PIPE LINE CO from 2007 to 2009.

	Was named BLACK LAKE PIPELINE from 2005 to 2006.

	Was named BLACK LAKE PIPELINE COMPANY from 2004 to 2004.



In [193]:
print(next(issues))


ASCEND PERFORMANCE MATERIALS TEXAS INC (OPERATOR_ID 31517) has changed its name:

	Was named ASCEND PERFORMANCE MATERIALS OPERATIONS LLC from 2012 to 2014.

	Was named ASCEND PERFORMANCE MATERIALS TEXAS INC from 2015 to 2016.

	Was named ASCEND PERFORMANCE MATERIALS, LLC from 2009 to 2011.

	Was named SOLUTIA INC from 2004 to 2005.

	Was named SOLUTIA, INC from 2006 to 2008.



In [194]:
print(next(issues))


SUNCOR ENERGY (USA) PIPELINE CO. (OPERATOR_ID 31822) has changed its name:

	Was named SUNCOR ENERGY (U.S.A.) PIPELINE COMPANY from 2006 to 2006.

	Was named SUNCOR ENERGY (USA) PIPELINE CO. from 2004 to 2018.

	Was named SUNCOR ENERGY (USA) PIPELINE COMPANY from 2005 to 2005.



In [195]:
print(next(issues))


WILLIAMS FIELD SERVICES - GULF COAST COMPANY, LP (OPERATOR_ID 994) has changed its name:

	Was named WILLIAMS FIELD SERVICES - GULF COAST COMPANT, L.P. from 2005 to 2005.

	Was named WILLIAMS FIELD SERVICES - GULF COAST COMPANY, L.P. from 2004 to 2006.

	Was named WILLIAMS FIELD SERVICES - GULF COAST COMPANY, LP from 2007 to 2018.



In [196]:
print(next(issues))


NAVAJO PIPELINE CO (OPERATOR_ID 13161) has changed its name:

	Was named NAVAJO PIPELINE CO from 2005 to 2007.

	Was named NAVAJO PIPELINE CO, LP from 2004 to 2004.

	Was named NAVAJO PIPELINE COMPANY from 2006 to 2006.

NAVAJO PIPELINE CO (OPERATOR_ID 13161) may have a namesake or namesakes:

	NAVAJO NATION OIL AND GAS COMPANY (OPERATOR_ID 31663)

	NAVAJO NATION OIL & GAS CO. (OPERATOR_ID 31663)

	NAVAJO NATION OIL & OIL GAS CO. (OPERATOR_ID 31663)

	NAVAJO NATION OIL & GAS (OPERATOR_ID 31663)



### 8.2.58 LDH

In [197]:
print(next(issues))


LDH ENERGY PIPELINE L.P. (OPERATOR_ID 32035) has changed its name:

	Was named LDH ENERGY PIPELINE L.P. from 2007 to 2010.

	Was named LOUIS DREYFUS PIPELINE L.P. from 2005 to 2006.

	Was named LOUIS DREYFUS PIPELINE LP  from 2004 to 2004.

LDH ENERGY PIPELINE L.P. (OPERATOR_ID 32035) has the same parent company as:

	LDH ENERGY OLEFINS LLC (OPERATOR_ID 31673)

	LDH ENERGY MONT BELVIEU L.P. (OPERATOR_ID 32246)

LDH ENERGY PIPELINE L.P. (OPERATOR_ID 32035) may have a namesake or namesakes:

	LDH ENERGY RESOURCES LLC (OPERATOR_ID 32246)

	BOARDWALK LOUISIANA MIDSTREAM, LLC (OPERATOR_ID 39138)

	LOUIS DREYFUS OLEFINS, LLC (OPERATOR_ID 31673)

	LOUIS DREYFUS OLEFINS LLC (OPERATOR_ID 31673)



In [198]:
report_names(['32035', '31673', '32246'])


Organization 32035 has changed its name:
    32035 was named Ldh Energy Pipeline LP from 2007 to 2010.
    32035 was named Louis Dreyfus Pipeline LP from 2005 to 2006.
    32035 was named Louis Dreyfus Pipeline Lp  from 2004 to 2004.

Organization 31673 has changed its name:
    31673 was named Ldh Energy Olefins Llc from 2007 to 2010.
    31673 was named Louis Dreyfus Olefins Llc from 2005 to 2006.


Our notes don't indicate an M&A has taken place.

In [199]:
company_groups = add_company_group('LDH Energy (Group)', ['32035', '31673', '32246'], company_groups)
company_groups.tail()

,members,name
132,31167,Torrance (Group)
133,39535,Torrance (Group)
134,32035,LDH Energy (Group)
135,31673,LDH Energy (Group)
136,32246,LDH Energy (Group)


According to our notes, LDH has been acquired by JV between Energy Transfer Partners and Regency Energy Partners, controlled by ETP on May 2, 2011.

In [200]:
m_as = add_m_a('Energy Transfer Partners (Group)', ['Energy Transfer Partners (Group)', 'LDH Energy (Group)'], m_as, start_year = '2011')
m_as.tail()

,members,name,start_year,end_year
32,22175,Targa (Group),2006,NaN
33,Boardwalk (Group),Boardwalk (Group),2014,NaN
34,31554,Boardwalk (Group),2014,NaN
35,Energy Transfer Partners (Group),Energy Transfer Partners (Group),2011,NaN
36,LDH Energy (Group),Energy Transfer Partners (Group),2011,NaN


In [201]:
print(next(issues))


MOTIVA ENTERPRISE LLC (OPERATOR_ID 30777) has changed its name:

	Was named MOTIVA ENTERPRISE LLC from 2004 to 2018.

	Was named MOTIVA ENTERPRISE LLC  from 2004 to 2004.

	Was named MOTIVA ENTERPRISES LLC from 2005 to 2005.

	Was named MOTIVA ENTERPRISES, LLC from 2006 to 2006.

	Was named Motiva Enterprises LLC from 2010 to 2010.

	Was named Motiva Enterprises LLC Convent Refinery from 2010 to 2010.



In [202]:
print(next(issues))


TEXAS EASTMAN DIVISION, EASTMAN CHEMICAL CO (OPERATOR_ID 26103) has changed its name:

	Was named TEXAS EASTMAN DIVISION from 2005 to 2006.

	Was named TEXAS EASTMAN DIVISION  from 2004 to 2004.

	Was named TEXAS EASTMAN DIVISION, EASTMAN CHEMICAL CO from 2007 to 2018.



In [203]:
print(next(issues))


COLLINS PIPELINE CO (OPERATOR_ID 26120) has changed its name:

	Was named COLLINS PIPELINE CO from 2007 to 2018.

	Was named COLLINS PIPELINE COMPANY from 2005 to 2006.

	Was named COLLINS PIPELINE LLC from 2004 to 2004.



In [204]:
print(next(issues))


HUNT CRUDE OIL SUPPLY CO (OPERATOR_ID 7660) has changed its name:

	Was named HUNT CRUDE OIL SUPPLY CO from 2007 to 2018.

	Was named HUNT CRUDE OIL SUPPLY COMPANY from 2004 to 2006.

HUNT CRUDE OIL SUPPLY CO (OPERATOR_ID 7660) may have a namesake or namesakes:

	HUNT CRUDE OIL SUPPLY COMPANY (OPERATOR_ID 26048)



In [205]:
print(next(issues))


ENLINK NGL PIPELINE, LP (OPERATOR_ID 32107) has changed its name:

	Was named CROSSTEX NGL PIPELINE, L.P. from 2006 to 2012.

	Was named ENLINK NGL PIPELINE, LP from 2013 to 2016.



In [206]:
print(next(issues))


TRANSMONTAIGNE OPERATING COMPANY L.P. (OPERATOR_ID 30909) has changed its name:

	Was named TRANSMONTAIGNE OPERATING COMPANY L.P. from 2007 to 2018.

	Was named TRANSMONTAIGNE PRODUCT SERVICES INC  from 2004 to 2004.

	Was named TRANSMONTAIGNE PRODUCT SERVICES INC. from 2005 to 2005.

	Was named TRANSMONTAIGNE PRODUCT SERVICES, INC. from 2006 to 2006.

	Was named TRANSMONTAIGNE PRODUCTS SERVICES INC from 2007 to 2008.



In [207]:
print(next(issues))


COPANO NGL SERVICES (MARKHAM), LLC (OPERATOR_ID 32541) may have a namesake or namesakes:

	COPANO NGL SERVICES, L.P. (OPERATOR_ID 32114)



In [208]:
print(next(issues))


MAGELLAN PIPELINES HOLDINGS, LP (OPERATOR_ID 31579) has changed its name:

	Was named MAGELLAN PIPELINES HOLDINGS, L.P. from 2005 to 2006.

	Was named MAGELLAN PIPELINES HOLDINGS, LP from 2004 to 2018.

	Was named Magellan Pipelines Holdings, L.P. from 2010 to 2010.

MAGELLAN PIPELINES HOLDINGS, LP (OPERATOR_ID 31579) may have a namesake or namesakes:

	Magellan Ammonia Pipeline, L.P. (OPERATOR_ID 12105)

	Magellan Pipeline Company, L.P. (OPERATOR_ID 22610)

	MAGELLAN PIPELINE COMPANY, LLC (OPERATOR_ID 22610)

	MAGELLAN PIPELINE COMPANY, L.P. (OPERATOR_ID 22610)



In [209]:
print(next(issues))


DELEK MARKETING AND SUPPLY, LP (OPERATOR_ID 15851) has changed its name:

	Was named DELEK MARKETING & SUPPLY, LP from 2006 to 2006.

	Was named DELEK MARKETING AND SUPPLY, LP from 2007 to 2018.

	Was named PRIDE REFINING, INC. from 2004 to 2005.

DELEK MARKETING AND SUPPLY, LP (OPERATOR_ID 15851) may have a namesake or namesakes:

	DELEK PIPELINE TEXAS, INC (OPERATOR_ID 26061)

	DELEK PIPELINE TEXAS, INC. (OPERATOR_ID 26061)

	DELEK US HOLDINGS (OPERATOR_ID 26061)

	DELEK U.S. HOLDINGS (OPERATOR_ID 26061)



In [210]:
print(next(issues))


DELEK CRUDE LOGISTICS, LLC. (OPERATOR_ID 26061) has changed its name:

	Was named DELEK CRUDE LOGISTICS, LLC. from 2013 to 2018.

	Was named DELEK PIPELINE TEXAS, INC from 2006 to 2012.

	Was named DELEK PIPELINE TEXAS, INC. from 2004 to 2007.

	Was named MCMURREY PIPELINE CO from 2008 to 2008.

DELEK CRUDE LOGISTICS, LLC. (OPERATOR_ID 26061) may have a namesake or namesakes:

	DELEK MARKETING & SUPPLY, LP (OPERATOR_ID 15851)



In [211]:
print(next(issues))


PREMCOR P A PIPELINE COMPANY (OPERATOR_ID 31415) may have a namesake or namesakes:

	THE PREMCOR REFINING GROUP, INC (OPERATOR_ID 31742)

	THE PREMCOR REFINING GROUP INC (OPERATOR_ID 31742)



In [212]:
print(next(issues))


TAMPA BAY PIPELINE CO. (OPERATOR_ID 26099) has changed its name:

	Was named TAMPA BAY PIPELINE CO. from 2007 to 2018.

	Was named TAMPA BAY PIPELINE COMPANY from 2005 to 2006.



In [213]:
print(next(issues))


GENESIS PIPELINE TEXAS, L.P. (OPERATOR_ID 32407) may have a namesake or namesakes:

	GENESIS PIPELINE USA, L.P. (OPERATOR_ID 31045)

	GENESIS PIPELINE TEXAS LP (OPERATOR_ID 31045)

	GENESIS CO2 PIPELINE, L. P. (OPERATOR_ID 31045)

	GENESIS CRUDE OIL LP (OPERATOR_ID 31045)



In [214]:
print(next(issues))


FRONT RANGE PIPELINE, LLC. (OPERATOR_ID 32283) has changed its name:

	Was named FRONT RANGE PIPELINE LLC from 2004 to 2004.

	Was named FRONT RANGE PIPELINE, LLC from 2005 to 2007.

	Was named FRONT RANGE PIPELINE, LLC. from 2008 to 2018.



In [215]:
print(next(issues))


GLASS MOUNTAIN PIPELINE (OPERATOR_ID 39080) has the same parent company as:

	GLASS MOUNTAIN PIPELINE, LLC (OPERATOR_ID 39774)



### 8.2.59 Glass Mountain

In [216]:
print(next(issues))


GLASS MOUNTAIN PIPELINE, LLC (OPERATOR_ID 39774) has changed its name:

	Was named GLASS MOUNTAIN PIPELINE, LLC from 2018 to 2018.

	Was named NAVIGATOR ENERGY SERVICES, LLC from 2017 to 2017.



In [217]:
find_total_miles('39080', range(2015, 2020))

[[2015, 72.5], [2016, 98.83], [2017, 0.0], [2018, 0.0], [2019, 0.0]]

In [218]:
find_total_miles('39774', range(2015, 2020))

[[2015, 0.0], [2016, 0.0], [2017, 98.83], [2018, 79.35], [2019, 0.0]]

In [219]:
report_names(['39080', '39774'])


Organization 39774 has changed its name:
    39774 was named Glass Mountain Pipeline Llc from 2018 to 2018.
    39774 was named Navigator Energy Services Llc from 2017 to 2017.


LexisNexis does not indicate who the owner of Glass Mountain is, but shows that Navigator Energy was acquired by NuStar on May 4, 2017 (Deal Number 3004179). Since the two IDs obviously refer to the same organization, and this organization did not then continue reporting as part of NuStar, there is nothing we need to do other than group them together.

In [220]:
company_groups = add_company_group('Glass Mountain (Group)', ['39080', '39774'], company_groups)
company_groups.tail()

,members,name
134,32035,LDH Energy (Group)
135,31673,LDH Energy (Group)
136,32246,LDH Energy (Group)
137,39080,Glass Mountain (Group)
138,39774,Glass Mountain (Group)


In [221]:
print(next(issues))


KINDER MORGAN LIQUID TERMINALS, LLC (OPERATOR_ID 26041) has changed its name:

	Was named ICPT, LLC from 2006 to 2006.

	Was named KINDER MORGAN LIQUID TERMINAL, LLC (WEST COAST TERMINALS) from 2005 to 2006.

	Was named KINDER MORGAN LIQUID TERMINALS LLC (WEST COAST TERMINALS) from 2005 to 2005.

	Was named KINDER MORGAN LIQUID TERMINALS, LLC from 2007 to 2018.

	Was named KINDER MORGAN LIQUIDS TERMINALS LLC from 2005 to 2005.

	Was named KINDER MORGAN LIQUIDS TERMINALS LLC (WEST COAST TERMINALS) from 2006 to 2006.

	Was named KINDER MORGAN LIQUIDS TERMINALS, L.P. from 2006 to 2006.

	Was named KINDER MORGAN LIQUIDS TERMINALS, LLC from 2004 to 2006.

	Was named KINDER MORGAN T. GABRIEL from 2005 to 2005.

	Was named KM LIQUIDS TERMINALS, L.P. from 2004 to 2005.

KINDER MORGAN LIQUID TERMINALS, LLC (OPERATOR_ID 26041) may have a namesake or namesakes:

	KINDER MORGAN ENERGY PARTNERS, L.P. (OPERATOR_ID 4472)

	KINDER MORGAN CANADA INC. (OPERATOR_ID 19585)

	KINDER MORGAN CANAADA INC. (O

In [222]:
print(next(issues))


ENBRIDGE PIPELINES (TOLEDO) INC (OPERATOR_ID 31448) has changed its name:

	Was named ENBRIDGE PIPELINES (TOLEDO) INC from 2007 to 2018.

	Was named ENBRIDGE PIPELINES (TOLEDO) INC  from 2004 to 2004.

	Was named ENBRIDGE PIPELINES (TOLEDO) INC. from 2005 to 2006.

ENBRIDGE PIPELINES (TOLEDO) INC (OPERATOR_ID 31448) may have a namesake or namesakes:

	ENBRIDGE ENERGY PARTNERS, LP (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, LP  (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 11169)

	ENBRIDGE ENERGY PARTNERS, L..P. (OPERATOR_ID 11169)

	ENBRIDGE PIPELINES (NORTH DAKOTA) LLC (OPERATOR_ID 15774)

	ENBRIDGE ENERGY PARTNERS, LP (OPERATOR_ID 15774)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 15774)

	ENBRIDGE ENERGY PARTNERS, LP (OPERATOR_ID 31947)

	ENBRIDGE ENERGY PARTNERS, L.P. (OPERATOR_ID 31947)

	ENBRIDGE ENERGY COMPANY INC. (OPERATOR_ID 32080)

	ENBRIDGE ENERGY COMPANY, INC. (OPERATOR_ID 32080)



In [223]:
print(next(issues))


NAVAJO NATION OIL AND GAS COMPANY (OPERATOR_ID 31663) has changed its name:

	Was named NAVAJO NATION OIL AND GAS COMPANY from 2007 to 2018.

	Was named RUNNING HORSE PIPELINE from 2004 to 2006.

NAVAJO NATION OIL AND GAS COMPANY (OPERATOR_ID 31663) may have a namesake or namesakes:

	NAVAJO PIPELINE CO (OPERATOR_ID 13161)

	NAVAJO PIPELINE COMPANY (OPERATOR_ID 13161)



In [224]:
print(next(issues))


PACIFIC MARKETING AND TRANSPORTATION LLC (OPERATOR_ID 31695) has changed its name:

	Was named PACIFIC MARKETING & TRANSPORTATION  LLC from 2006 to 2006.

	Was named PACIFIC MARKETING AND TRANSPORTATION LLC from 2005 to 2007.

	Was named PACIFIC MARKETING AND TRANSPORTATION LLC  from 2004 to 2004.

PACIFIC MARKETING AND TRANSPORTATION LLC (OPERATOR_ID 31695) may have a namesake or namesakes:

	MOBIL PACIFIC PIPELINE CO (OPERATOR_ID 30005)

	MOBIL PACIFIC PIPELINE COMPANY (OPERATOR_ID 30005)

	MOBIL PACIFIC PIPELINE (OPERATOR_ID 30005)

	PACIFIC ENERGY PARTNER, LP (OPERATOR_ID 5656)

	PACIFIC ENERGY GROUP, LLC (OPERATOR_ID 5656)

	PACIFIC ENERGY PARTNER, LP (OPERATOR_ID 30968)

	PACIFIC ENERGY GROUP, LLC (OPERATOR_ID 30968)

	PACIFIC ENERGY GROUP LLC  (OPERATOR_ID 31325)

	PACIFIC ENERGY PARTNER, LP (OPERATOR_ID 31325)

	PACIFIC ENERGY GROUP, LLC (OPERATOR_ID 31325)

	PACIFIC ENERGY PARTNERS, LP (OPERATOR_ID 31666)

	PACIFIC ENERGY PARTNER, LP (OPERATOR_ID 31666)

	PACIFIC ENERGY GROUP

In [225]:
print(next(issues))


KINDER MORGAN WINK PIPELINE LLC (OPERATOR_ID 31957) has changed its name:

	Was named KINDER MORGAN WINK PIPELINE LLC from 2010 to 2018.

	Was named KINDER MORGAN WINK PIPELINE LP from 2004 to 2014.

	Was named KINDER MORGAN WINK PIPELINE, LP from 2006 to 2006.

KINDER MORGAN WINK PIPELINE LLC (OPERATOR_ID 31957) may have a namesake or namesakes:

	KINDER MORGAN ENERGY PARTNERS, L.P. (OPERATOR_ID 4472)

	KINDER MORGAN CANADA INC. (OPERATOR_ID 19585)

	KINDER MORGAN CANAADA INC. (OPERATOR_ID 19585)

	KINDER MORGAN CANADA INC.  (OPERATOR_ID 19585)

	KINDER MORGAN LIQUID TERMINALS, LLC (OPERATOR_ID 26041)

	KINDER MORGAN, INC (OPERATOR_ID 26041)

	KINDER MORGAN LIQUID TERMINAL, LLC (WEST COAST TERMINALS) (OPERATOR_ID 26041)

	KINDER MORGAN LIQUIDS TERMINALS LLC (OPERATOR_ID 26041)

	KINDER MORGAN LIQUIDS TERMINALS, L.P. (OPERATOR_ID 26041)

	KINDER MORGAN PIPELINES (USA) INC (OPERATOR_ID 31720)

	KINDER MORGAN CANADA INC. (OPERATOR_ID 31720)

	KINDER MORGAN CANADA, INC. (OPERATOR_ID 3172

In [226]:
print(next(issues))


HARBOR PIPELINE CO (OPERATOR_ID 7063) has changed its name:

	Was named HARBOR PIPELINE from 2006 to 2006.

	Was named HARBOR PIPELINE (OPERATED BY SUNOCO PIPELINE L.P.) from 2004 to 2005.

	Was named HARBOR PIPELINE CO from 2007 to 2018.



In [227]:
print(next(issues))


PACIFIC PIPELINE SYSTEM LLC (OPERATOR_ID 31885) has changed its name:

	Was named PACIFIC PIPELINE SYSTEM LLC from 2005 to 2010.

	Was named PACIFIC TERMINALS  LLC from 2006 to 2006.

	Was named PACIFIC TERMINALS LLC from 2007 to 2008.

	Was named PACIFIC TERMINALS LLC  from 2004 to 2004.



In [228]:
print(next(issues))


OILTANKING, HOUSTON LP (OPERATOR_ID 14194) has changed its name:

	Was named OIL TANKING HOUSTON, L.P. from 2005 to 2005.

	Was named OILTANKING HOUSTON, L.P. from 2006 to 2006.

	Was named OILTANKING HOUSTON, LP from 2004 to 2004.

	Was named OILTANKING, HOUSTON LP from 2007 to 2018.



In [229]:
print(next(issues))


WESTERN REFINING SOUTHWEST, INC (OPERATOR_ID 2371) has changed its name:

	Was named CINIZA PIPE LINE from 2004 to 2004.

	Was named CINIZA PIPE LINE  from 2005 to 2005.

	Was named CINIZA PIPE LINE COMPANY from 2006 to 2006.

	Was named CINIZA PIPELINE  from 2005 to 2005.

	Was named WESTERN REFINING SOUTHWEST, INC from 2007 to 2017.

WESTERN REFINING SOUTHWEST, INC (OPERATOR_ID 2371) may have a namesake or namesakes:

	WESTERN REFINING PIPELINE, LLC (OPERATOR_ID 31574)

	WESTERN REFINING PIPELINE COMPANY (OPERATOR_ID 31574)

	WESTERN REFINING LOGISTICS, LP (OPERATOR_ID 31574)

	WESTERN REFINING INC. (OPERATOR_ID 31574)



In [230]:
print(next(issues))


CITGO REFINING & CHEMICAL CO. L.P. (OPERATOR_ID 31023) may have a namesake or namesakes:

	CITGO PETROLEUM  (OPERATOR_ID 2387)

	CITGO PETROLEUM (OPERATOR_ID 2387)

	CITGO PETROLEUM (OPERATOR_ID 30755)



In [231]:
print(next(issues))


COPANO NGL SERVICES LLC (OPERATOR_ID 32114) has changed its name:

	Was named COPANO NGL SERVICES LLC from 2013 to 2018.

	Was named COPANO NGL SERVICES, L.P. from 2005 to 2012.



In [232]:
print(next(issues))


ONEOK FIELD SERVICES  (OPERATOR_ID 30629) may have a namesake or namesakes:

	ONEOK NGL PIPELINE LP (OPERATOR_ID 32109)

	ONEOK INC (OPERATOR_ID 32109)



In [233]:
print(next(issues))


LDH ENERGY OLEFINS LLC (OPERATOR_ID 31673) has changed its name:

	Was named LDH ENERGY OLEFINS LLC from 2007 to 2010.

	Was named LOUIS DREYFUS OLEFINS LLC from 2006 to 2006.

	Was named LOUIS DREYFUS OLEFINS, LLC from 2005 to 2005.

LDH ENERGY OLEFINS LLC (OPERATOR_ID 31673) may have a namesake or namesakes:

	LDH ENERGY RESOURCES LLC (OPERATOR_ID 32246)

	BOARDWALK LOUISIANA MIDSTREAM, LLC (OPERATOR_ID 39138)

	LOUIS DREYFUS PIPELINE LP  (OPERATOR_ID 32035)

	LOUIS DREYFUS PIPELINE L.P. (OPERATOR_ID 32035)



In [234]:
print(next(issues))


OSAGE PIPE LINE COMPANY, LLC (OPERATOR_ID 14391) has changed its name:

	Was named OSAGE PIPE LINE COMPANY, LLC from 2007 to 2018.

	Was named OSAGE PIPELINE COMPANY from 2004 to 2004.

	Was named OSAGE PIPELINE COMPANY, LLC from 2005 to 2006.

	Was named Osage Pipeline Company, LLC from 2010 to 2010.



In [235]:
print(next(issues))


CHEVRON PRODUCTS COMPANY - HAWAII (OPERATOR_ID 2339) has changed its name:

	Was named CHEVRON PRODUCTS COMPANY - HAWAII from 2010 to 2015.

	Was named CHEVRON PRODUCTS COMPANY - HAWAII  from 2007 to 2009.

	Was named CHEVRON PRODUCTS COMPANY HAWAII REFINERY from 2004 to 2005.

	Was named CHEVRON PRODUCTS COMPANY, HAWAII REFINERY from 2006 to 2006.

CHEVRON PRODUCTS COMPANY - HAWAII (OPERATOR_ID 2339) may have a namesake or namesakes:

	CHEVRON PIPELINE COMPANY (OPERATOR_ID 2731)

	CHEVRON PIPE LINE COMPANY (OPERATOR_ID 2731)

	CHEVRON PETROCHEMICAL PIPELINE LLC (OPERATOR_ID 31554)

	CHEVRON PETROCHEMICAL PIPELINE LLC (FORMERLY TEXACO PETROCHEMICAL PIPELINE LLC) (OPERATOR_ID 31554)



In [236]:
print(next(issues))


TESORO LOGISTICS OPERATIONS LLC - MOUNTAIN REGION (OPERATOR_ID 31583) has changed its name:

	Was named TESORO LOGISTICS OPERATIONS LLC - MOUNTAIN REGION from 2012 to 2017.

	Was named TESORO REFINING & MARKETING CO, MOUNTAIN REGION  from 2004 to 2004.

	Was named TESORO REFINING & MARKETING CO., MOUNTAIN REGION from 2007 to 2011.

	Was named TESORO REFINING & MARKETING CO.; MOUNTAIN REGION from 2005 to 2006.

TESORO LOGISTICS OPERATIONS LLC - MOUNTAIN REGION (OPERATOR_ID 31583) may have a namesake or namesakes:

	TESORO - HIGH PLAINS PIPELINE COMPANY (OPERATOR_ID 31570)

	TESORO HIGH PLAINS PIPELINE COMPANY  (OPERATOR_ID 31570)

	TESORO CORPORATION (OPERATOR_ID 31570)

	TESORO REFINING & MARKETING C O., GOLDEN EAGLE (OPERATOR_ID 31874)

	TESORO CORPORATION (OPERATOR_ID 31874)

	TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA (OPERATOR_ID 38933)



In [237]:
print(next(issues))


BOARDWALK PETROCHEMICAL PIPELINE, LLC (OPERATOR_ID 31554) has changed its name:

	Was named BOARDWALK PETROCHEMICAL PIPELINE, LLC from 2014 to 2018.

	Was named CHEVRON PETROCHEMICAL PIPELINE LLC from 2007 to 2013.

	Was named CHEVRON PETROCHEMICAL PIPELINE LLC (FORMERLY TEXACO PETROCHEMICAL PIPELINE LLC) from 2006 to 2006.

	Was named TEXACO PETROCHEMICAL PIPELINE LLC from 2004 to 2005.

BOARDWALK PETROCHEMICAL PIPELINE, LLC (OPERATOR_ID 31554) may have a namesake or namesakes:

	CHEVRON PIPE LINE CO (OPERATOR_ID 2731)

	CHEVRON PIPELINE COMPANY (OPERATOR_ID 2731)

	CHEVRON PIPE LINE COMPANY (OPERATOR_ID 2731)

	CHEVRON PRODUCTS COMPANY - HAWAII (OPERATOR_ID 2339)

	CHEVRON USA, INC. (OPERATOR_ID 2339)

	CHEVRON USA,  INC. (OPERATOR_ID 2339)

	CHEVRON PRODUCTS COMPANY - HAWAII  (OPERATOR_ID 2339)

	CHEVRON MIDSTREAM PIPELINES LLC (OPERATOR_ID 31556)

	TEXACO PIPELINES LLC (OPERATOR_ID 31556)

	TEXACO PIPELINES, LLC (OPERATOR_ID 31556)



In [238]:
print(next(issues))


CITGO PIPELINE CO (OPERATOR_ID 2387) has changed its name:

	Was named CITGO PIPELINE from 2006 to 2006.

	Was named CITGO PIPELINE CO from 2007 to 2018.

	Was named CITGO PIPELINE COMPANY from 2004 to 2005.

CITGO PIPELINE CO (OPERATOR_ID 2387) may have a namesake or namesakes:

	CITGO PETROLEUM (OPERATOR_ID 30755)

	CITGO PETROLEUM CORPORATION (OPERATOR_ID 31023)

	CITGO PETROLEUM COMPANY (OPERATOR_ID 31023)



In [239]:
print(next(issues))


TRANS MOUNTAIN PIPELINE (PUGET SOUND) LLC (OPERATOR_ID 19585) has changed its name:

	Was named TERASEN PIPELINE (PUGET SOUND) CORP. from 2007 to 2007.

	Was named TERASEN PIPELINES (PUGET SOUND) CORP. from 2005 to 2006.

	Was named TERASEN PIPELINES (PUGET SOUND) INC from 2004 to 2004.

	Was named TRANS MOUNTAIN PIPELINE (PUGET SOUND) LLC from 2008 to 2018.

TRANS MOUNTAIN PIPELINE (PUGET SOUND) LLC (OPERATOR_ID 19585) may have a namesake or namesakes:

	TERASEN INC  (OPERATOR_ID 31720)



In [240]:
print(next(issues))


MOBIL PACIFIC PIPELINE CO (OPERATOR_ID 30005) has changed its name:

	Was named MOBIL PACIFIC PIPELINE from 2005 to 2005.

	Was named MOBIL PACIFIC PIPELINE CO from 2007 to 2015.

	Was named MOBIL PACIFIC PIPELINE COMPANY from 2004 to 2006.

MOBIL PACIFIC PIPELINE CO (OPERATOR_ID 30005) may have a namesake or namesakes:

	EXXONMOBIL PIPELINE COMPANY (OPERATOR_ID 4906)

	MOBIL CORP (OPERATOR_ID 12624)

	MOBIL OIL CORPORATION - MOBIL BEAUMNT REFINERY (HULL) (OPERATOR_ID 12624)

	MOBIL OIL  CORPORATION-MOBIL BEAUMONT REFINERY (MAGPETCO) (OPERATOR_ID 12624)

	MOBIL OIL CORPORATION - MOBIL BEAUMONT REFINERY (HULL) (OPERATOR_ID 12624)

	EXXONMOBIL OIL CORPORATION - BEAUMONT REFINERY (HULL) (OPERATOR_ID 12624)

	EXXONMOBIL OIL CORPORATION - BEAUMONT REFINERY (MAGPETCO) (OPERATOR_ID 12624)

	MOBIL BEAUMONT REFINERY (OPERATOR_ID 12624)

	MOBIL  PIPE  LINE COMPANY (OPERATOR_ID 12628)

	MOBIL PIPELINE CO (OPERATOR_ID 12628)

	MOBIL PIPE LINE COMPANY (OPERATOR_ID 12628)

	MOBIL CHEMICAL CO (OPERA

In [241]:
print(next(issues))


MARKWEST MICHIGAN PIPELINE COMPANY, L.L.C. (OPERATOR_ID 31871) has changed its name:

	Was named MARKWEST MICHIGAN PIPELINE COMPANY L.L.C from 2006 to 2006.

	Was named MARKWEST MICHIGAN PIPELINE COMPANY, L.L.C. from 2018 to 2018.

	Was named MARKWEST MICHIGAN PIPELINE, LLC from 2004 to 2017.

	Was named MARKWEST-MICHIGAN PIPELINE COMPANY, L.L.C from 2005 to 2005.

MARKWEST MICHIGAN PIPELINE COMPANY, L.L.C. (OPERATOR_ID 31871) may have a namesake or namesakes:

	MARKWEST RANGER PIPELINE COMPANY, LLC (OPERATOR_ID 26026)

	MARKWEST ENERGY APPALACHIA, LLC (OPERATOR_ID 26026)

	MARKWEST HYDROCARBON, INC (OPERATOR_ID 26026)

	MARKWEST HYDROCARBON, INC. (OPERATOR_ID 26026)

	MARKWEST ENERGY PARTNERS, L.P. (OPERATOR_ID 26026)

	MARKWEST LIBERTY MIDSTREAM & RESOURCES, LLC (OPERATOR_ID 32412)

	MARKWEST LIBERTY MIDSTREAM & RESOURCES, L.L.C. (OPERATOR_ID 32412)



In [242]:
print(next(issues))


TESORO LOGISTICS OPERATIONS LLC - GOLDEN EAGLE (OPERATOR_ID 31874) has changed its name:

	Was named TESORO LOGISTICS OPERATIONS LLC - GOLDEN EAGLE from 2012 to 2017.

	Was named TESORO REFINING & MARKETING C O., GOLDEN EAGLE from 2007 to 2011.

	Was named TESORO REFINING & MARKETING C O., GOLDEN EAGLE  from 2004 to 2004.

	Was named TESORO REFINING & MARKETING CO  from 2004 to 2004.

	Was named TESORO REFINING & MARKETING CO., GOLDEN EAGLE  from 2006 to 2006.

	Was named TESORO REFINING & MARKETING CO.; GOLDEN EAGLE from 2005 to 2005.

TESORO LOGISTICS OPERATIONS LLC - GOLDEN EAGLE (OPERATOR_ID 31874) may have a namesake or namesakes:

	TESORO - HIGH PLAINS PIPELINE COMPANY (OPERATOR_ID 31570)

	TESORO HIGH PLAINS PIPELINE COMPANY  (OPERATOR_ID 31570)

	TESORO CORPORATION (OPERATOR_ID 31570)

	TESORO REFINING & MARKETING CO., MOUNTAIN REGION (OPERATOR_ID 31583)

	TESORO REFINING & MARKETING CO, MOUNTAIN REGION  (OPERATOR_ID 31583)

	TESORO CORPORATION (OPERATOR_ID 31583)

	TESORO LOG

In [243]:
print(next(issues))


INTERSTATE ENERGY COMPANY (OPERATOR_ID 8166) has changed its name:

	Was named INTERSTATE ENERGY COMPANY from 2016 to 2018.

	Was named PPL INTERSTATE ENERGY CO from 2010 to 2015.

	Was named PPL INTERSTATE ENERGY CO  from 2007 to 2009.

	Was named PPL INTERSTATE ENERGY COMPANY from 2004 to 2006.



In [244]:
print(next(issues))


DOW PIPELINE CO (OPERATOR_ID 3535) has changed its name:

	Was named DOW PIPELINE CO from 2007 to 2018.

	Was named DOW PIPELINE COMPANY from 2004 to 2006.

DOW PIPELINE CO (OPERATOR_ID 3535) may have a namesake or namesakes:

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 2162)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 2162)

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 3527)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 3527)

	THE DOW CHEMICAL COMPANY, INCORPORATED (OPERATOR_ID 26086)

	THE DOW CHEMICAL COMPANY, INC. (OPERATOR_ID 26086)

	TARGA DOWNSTREAM LLC (OPERATOR_ID 39823)



In [245]:
print(next(issues))


NUSTAR TERMINALS OPERATIONS PARTNERSHIP L. P. (OPERATOR_ID 26094) has changed its name:

	Was named NUSTAR TERMINALS OPERATIONS PARTNERSHIP L. P. from 2007 to 2018.

	Was named SUPPORT TERMINALS OPERATING PARTNERSHIP from 2004 to 2004.

	Was named SUPPORT TERMINALS OPERATING PARTNERSHIP L.P. from 2005 to 2005.

	Was named SUPPORT TERMINALS OPERATING PARTNERSHIP, L.P. from 2006 to 2006.



In [246]:
print(next(issues))


CALIFORNIA RESOURCES ELK HILLS, LLC (OPERATOR_ID 31228) has changed its name:

	Was named CALIFORNIA RESOURCES ELK HILLS, LLC from 2014 to 2018.

	Was named OCCIDENTAL OF ELK HILLS, INC from 2007 to 2013.

	Was named OCCIDENTAL OF ELK HILLS, INC. from 2004 to 2006.

CALIFORNIA RESOURCES ELK HILLS, LLC (OPERATOR_ID 31228) may have a namesake or namesakes:

	TESORO LOGISTICS OPERATIONS LLC - SOUTHERN CALIFORNIA (OPERATOR_ID 38933)

	OCCIDENTAL PETROLEUM CORP (OPERATOR_ID 31888)



In [247]:
print(next(issues))

StopIteration: 

## 8.3 Overview

In [248]:
company_groups

,members,name
0,30829,Teppco (Group)
1,32209,Teppco (Group)
2,3445,Teppco (Group)
3,32109,ONEOK (Group)
4,30629,ONEOK (Group)
5,15485,Phillips 66 (Group)
6,31684,Phillips 66 (Group)
7,22610,Magellan (Group)
8,12105,Magellan (Group)
9,31579,Magellan (Group)


In [249]:
m_as

,members,name,start_year,end_year
0,Enterprise Products (Group),Enterprise Products (Group),2010,NaN
1,Teppco (Group),Enterprise Products (Group),2010,NaN
2,Sunoco (Group),Sunoco (Group),2011,NaN
3,32683,Sunoco (Group),2011,NaN
4,22442,Sunoco (Group),2011,NaN
5,Sunoco (Group),Sunoco (Group),2013,NaN
6,Energy Transfer Partners (Group),Sunoco (Group),2013,NaN
7,NuStar (Group),NuStar (Group),2007,NaN
8,10012,NuStar (Group),2007,NaN
9,NuStar (Group),NuStar (Group),2006,NaN


In [250]:
spin_offs

[]

## 8.4 Additions from ownership changes text file

In [251]:
find_name('39029')

[{'name': 'Tesoro Logistics Northwest Pipeline Llc',
  'start_year': 2013,
  'end_year': 2017,
  'id_': '39029'}]

Was sold by Chevron to Tesoro in 2013, but Tesoro set up a new unit for this pipeline, so we do not need to take any action.

In [252]:
company_groups.to_feather(f'../preprocessed_data/company_groups_{today}.feather')
m_as.to_feather(f'../preprocessed_data/m_as_{today}.feather')